In [62]:
from community import community_louvain
import gc
import logging
import os
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_probability as tfp
import warnings
logger = tf.get_logger()

os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [63]:
logging.basicConfig(encoding='utf-8', level=logging.DEBUG,
                    force = True)

In [64]:
def graph1_adjacency_matrix():
    return np.array([
        [0, 1, 1, 0, 0, 0, 1, 0, ],
        [1, 0, 0, 1, 0, 0, 0, 1, ],
        [1, 0, 0, 1, 1, 0, 0, 0, ],
        [0, 1, 1, 0, 0, 1, 0, 0, ],
        [0, 0, 1, 0, 0, 1, 1, 0, ],
        [0, 0, 0, 1, 1, 0, 0, 1, ],
        [1, 0, 0, 0, 1, 0, 0, 1, ],
        [0, 1, 0, 0, 0, 1, 1, 0, ],
    ])


In [65]:
def build_test_graph1():
    adj_mat = graph1_adjacency_matrix()
    N = len(adj_mat)

    G = nx.Graph()
    for i in range(N):
        G.add_node(i)

    for i in range(N):
        for j in range(N):
            if adj_mat[i][j] == 1:
                G.add_edge(i, j)
                G.add_edge(j, i)
    return G

In [81]:
def calc_W(G, C):
    n = len(G.nodes())
    m = len(G.edges())
    m_c = len(C.edges())
    W = np.zeros((n,n+m+m_c),dtype= 'int8')
    for i in range(n):
        W[i][i] = 1
    j = n
    for edge in G.edges():
        x = int(edge[0])
        y = int(edge[1])
        W[x][j] = 1
        W[y][j] = 1
        j+=1
    for edge in C.edges():
        x = int(edge[0])
        y = int(edge[1])
        W[x][j] = 1
        W[y][j] = 1
        j+=1
    return W

def calc_b(G, C):
    n = len(G.nodes())
    m = len(G.edges())
    m_c = len(C.edges())
    b = np.array([-1/2 for i in range(n)])
    b_m = np.array([-1 for i in range(m+m_c)])
    return np.concatenate((b,b_m))

def calc_w(G, C):
    n = len(G.nodes())
    m = len(G.edges())
    m_c = len(C.edges())
    w = np.array([-1 for i in range(n)])
    w_n = np.array([n for i in range(m)])
    w_c = np.array([-1 for i in range(m_c)])
    return np.concatenate((w,w_n,w_c))

def build_omega(G):
    n = len(G.nodes())
    max_degree = max([G.degree(i) for i in range(n)])
#     logging.debug("Max degree = {}".format(max_degree))
    if max_degree == 0:
        return np.array([1 for i in range(n)])
    return np.array([1 - G.degree(i)/max_degree + random.random()/1000 for i in range(n)])  # Works bad for sall graphs
#     return np.array([random.random() for i in range(n)])

def build_network(G):
    logging.info("Build network on graph G:{}.".format(str(G)))
    C = nx.complement(G)
    W = calc_W(G, C)
    b = calc_b(G, C)
    w = calc_w(G, C)
    omega = build_omega(G)
#     logging.debug("Initial omega = {}".format(omega))
    return (W,b,w, omega, C)


In [67]:
def get_result_nodes(theta, alpha = 0.5):
    return set(np.argwhere(theta.numpy() > alpha).reshape(-1))

def not_connected_nodes_exist_in_G(G, result_nodes):
    for v in G.nodes():
        if v not in result_nodes:
            node_not_connected_to_G = True
            for edge in G.neighbors(v):
                if edge in result_nodes:
                    node_not_connected_to_G = False
                    break
            if node_not_connected_to_G:
                return True
    return False
            
def graph_has_no_edges(G, result_nodes):
    for v in result_nodes:
        for edge in G.neighbors(v):
            if edge in result_nodes:
                return False
    return True
            
def result_is_valid(G, theta):
    result_nodes = get_result_nodes(theta)
    return graph_has_no_edges(G, result_nodes) and not not_connected_nodes_exist_in_G(G, result_nodes)
    
            
def train_network(G, max_epochs = 100):
    (W,b,w, theta, C) = build_network(G)
    previous_theta = theta
    n = len(G.nodes())
    W_t = tf.constant(W.T, dtype = 'float32')
    b = tf.constant(b, dtype = 'float32')
    w_t = tf.constant(w.T, dtype = 'float32')
    theta = tf.Variable(theta,
                        trainable=True,
                        constraint = lambda x: tf.clip_by_value(theta,0,1),
                        dtype = 'float32')
    e_n = tf.constant(np.ones((n)),dtype = 'float32')

    def network():
        h = tf.math.multiply(e_n,theta)
        h = tf.linalg.matvec(W_t,h)
        h = tf.add(h,b)
        h = tf.nn.relu(h)
        h = tf.tensordot(w_t,h, 1)
        return h

    def loss():
        h = network()
        return (h-h_d)**2
    

    h_d = -n*n/2
    optimizer=tf.optimizers.Adam(learning_rate=0.1,)
    
    epoch = 0
    while not result_is_valid(G, theta) and epoch < max_epochs:
        logging.debug("Epoch = {}".format(epoch))
        optimizer.minimize(loss, var_list=[theta])
#         logging.debug("theta = {}".format(theta))
        previous_error = loss()
#         logging.debug("Loss = {}".format(previous_error))
        epoch+=1
        if (previous_theta == theta.numpy()).all():
            logging.debug("Solver stuck")
            break
        previous_theta = theta.numpy()
    if result_is_valid(G, theta):
        logging.debug("Optimal solution found. Cardinality = {}".format(len(get_result_nodes(theta))))
    logging.debug("Total epochs = {}".format(epoch))
    return theta


In [68]:
def graphSets(graph): # Determinative algorithm to use correctness of the solution. Works too long for graphs >= 30 nodes.
      
    # Base Case - Given Graph 
    # has no nodes
    if(len(graph) == 0):
        return []
     
    # Base Case - Given Graph
    # has 1 node
    if(len(graph) == 1):
        return [list(graph.keys())[0]]
      
    # Select a vertex from the graph
    vCurrent = list(graph.keys())[0]
      
    # Case 1 - Proceed removing
    # the selected vertex
    # from the Maximal Set
    graph2 = dict(graph)
      
    # Delete current vertex 
    # from the Graph
    del graph2[vCurrent]
      
    # Recursive call - Gets 
    # Maximal Set,
    # assuming current Vertex 
    # not selected
    res1 = graphSets(graph2)
      
    # Case 2 - Proceed considering
    # the selected vertex as part
    # of the Maximal Set
  
    # Loop through its neighbours
    for v in graph[vCurrent]:
          
        # Delete neighbor from 
        # the current subgraph
        if(v in graph2):
            del graph2[v]
      
    # This result set contains VFirst,
    # and the result of recursive
    # call assuming neighbors of vFirst
    # are not selected
    res2 = [vCurrent] + graphSets(graph2)
      
    # Our final result is the one 
    # which is bigger, return it
    if(len(res1) > len(res2)):
        return res1
    return res2

def calculate_recursive_broot_force_result(G, draw_graph = True):
    dict_G = nx.to_dict_of_lists(G)
    print("Recursive approach")
    MIS = graphSets(dict_G)
    print(MIS)
    print("Length = "+str(len(MIS)))
    if draw_graph:
        color_map = ['red' if node in MIS else 'blue' for node in G]
        graph = nx.draw(G, node_color=color_map) # node lables
        plt.show()

In [69]:
def calculate_determenistic_result(G, draw_graph = True):
    print("Deterministic built-in approach")
    MIS = nx.maximal_independent_set(G)
    print(MIS)
    print("Length = "+str(len(MIS)))
    if draw_graph:
        color_map = ['yellow' if node in MIS else 'blue' for node in G]
        graph = nx.draw(G, node_color=color_map) # node lables
        plt.show()

In [70]:
def calculate_DNN_result(G, omega, threshold = 0.5, draw_graph = True):
    print("DNN approach")
    result_nodes = get_result_nodes(omega)
    print(result_nodes)
    print("Length = "+str(len(result_nodes)))
    if draw_graph:
        color_map = ['green' if node in result_nodes else 'blue' for node in G]        
        graph = nx.draw(G, node_color=color_map) # node lables
        plt.show()

In [71]:
def compareResults(G, max_epochs = 100, draw_graph = False):
#     calculate_recursive_broot_force_result(G)
    omega = train_network(G, max_epochs)
    calculate_determenistic_result(G, draw_graph)
    calculate_DNN_result(G, omega, draw_graph=draw_graph)

In [72]:
def read_G_from_file(filepath):
    return nx.read_edgelist(filepath)

In [12]:
# compareResults(build_test_graph1(), 100, True)

In [13]:
# compareResults(nx.complete_graph(10), draw_graph=False) # Expected result = 1

In [14]:
# compareResults(nx.graph_atlas(1000),draw_graph=True)

In [15]:
# compareResults(nx.gnp_random_graph(20, 0.1), 200, True) #Expected result around 12-14

In [16]:
# compareResults(nx.gnp_random_graph(50, 0.075), max_epochs = 1000)

In [17]:
# compareResults(nx.gnp_random_graph(100, 0.1), max_epochs = 1000, draw_graph=False)

In [13]:
wiki_vote_G = read_G_from_file('datasets/Wiki-Vote.txt') 

In [73]:
print(wiki_vote_G)

NameError: name 'wiki_vote_G' is not defined

In [74]:
def find_inter_cluster_edges(G, communities):
    edges = dict()
    for com in communities:
        for node_i in communities[com]:
            for neighbour_i in G.neighbors(node_i):
                if neighbour_i not in communities[com]:
                    if edges.get(node_i) is not None:
                        edges[node_i].append(neighbour_i)
                    else:
                        edges[node_i] = [neighbour_i]
                    if edges.get(neighbour_i) is not None:
                        edges[neighbour_i].append(node_i)
                    else:
                        edges[neighbour_i] = [node_i]
    return edges

def find_forbidden_edges(G, R, independent_sets):
    forbidden = []
    for node, edges in R.items():
        if node in independent_sets:
            for edge in edges:
                if edge in independent_sets:
                    forbidden.append((node, edge))
    return forbidden

def collect_list_by_dicts_key(partitions):
    communities = {}
    for key, val in partitions.items():
        if communities.get(val) == None:
            communities[val] = [key]
        else:
            communities[val].append(key)
    return communities

def build_G_from_nodes(G, nodes):
    communities = {}
    N = len(nodes)
    new_G = nx.Graph()
    index_map = dict()
    node_map = dict()
    index = 0
    for node in nodes:
        index_map[index] = node
        node_map[node] = index
        new_G.add_node(index)
        index+=1

    for i in nodes:
        for j in nodes:
            if G.has_edge(i,j):
                new_G.add_edge(node_map[i], node_map[j])
                new_G.add_edge(node_map[j], node_map[i])
    return (new_G, index_map, node_map)

In [75]:
def node_is_new_candidate(G, node, mis):
    neighbors = G.neighbors(node)
    neighbours_in_mis_count = 0
    for w in neighbors:
        if w in mis:
            neighbours_in_mis_count += 1
            if neighbours_in_mis_count == 2:
                break
            if neighbours_in_mis_count == 1:
                return (True, w)
    return (False, -1) 

def get_node_with_most_occurences(F, q, v):
    count_q = 0
    count_v = 0
    for x,y in F:
        if x == q:
            count_q+=1
        if y == v:
            count_v+=1
    if count_q > v:
        return q
    else:
        return v
            
def replace_node_if_possible(G,F,mis,node):
    if node not in mis:
        return True
    (can_be_replaced, new_node) = node_is_new_candidate(G,node,mis)
    if can_be_replaced:
        mis.remove(node)
        mis.add(new_node)
        return True
    return False 
        
def replace_forbiden_nodes(G,R,F,mis):
    logging.info("Replacing forbiden nodes")
    while len(F) > 0:
        logging.debug("Length of F = {}".format(len(F)))
        for q,v in F:
            replaced = replace_node_if_possible(G,F,mis,q)
            if replaced:
                break
            else:
                replaced = replace_node_if_possible(G,F,mis,v)
            if replaced:
                break
            else:
                node_to_be_removed = get_node_with_most_occurences(F,q,v)
                mis.remove(node_to_be_removed)
        F = find_forbidden_edges(G, R, mis)
    return mis

def build_G_from_left_nodes(G, mis):
    mis_with_neighbours = set()
    for node in mis:
        mis_with_neighbours.add(node)
        for neighbour in G.neighbors(node):
            mis_with_neighbours.add(neighbour)
    nodes_left_to_process = set(G.nodes()).difference(mis_with_neighbours)
    return build_G_from_nodes(G,nodes_left_to_process)

def calculate_mis_with_left_nodes(G, mis_list, max_epochs):
    (left_G, left_index_map, left_node_map)  = build_G_from_left_nodes(G, mis_list)
    logging.info("Left nodes size = {}".format(len(left_G.nodes())))
    if len(left_G.nodes()) > 1000:
        mis = calculate_large_G(left_G, max_epochs)
    else:
        mis = get_result_nodes(train_network(left_G, max_epochs))
    mis_correct = [left_index_map[node] for node in mis]
    mis_final = mis_list.union(mis)
    return mis_final

In [76]:
def calculate_large_G(G, max_epochs = 1000):
    gc.collect()
    partitions = community_louvain.best_partition(G, resolution= 0.8)
    communities = collect_list_by_dicts_key(partitions)
    logging.info("Total communities {}".format(len(communities)))
    R = find_inter_cluster_edges(G, communities)
    mis_list = set()
    community_index = 1
    for com in communities:
        logging.info("Community {} processing...".format(community_index))
        (small_G,index_map, node_map) = build_G_from_nodes(G, communities[com])
        omega_for_small_G = train_network(small_G, max_epochs)
        mis = get_result_nodes(omega_for_small_G)
        mis_correct = [index_map[node] for node in mis]
        mis_list = mis_list.union(mis_correct)
        community_index += 1
    F = find_forbidden_edges(G, R, mis_list)
    replace_forbiden_nodes(G,R,F,mis_list)
    return calculate_mis_with_left_nodes(G, mis_list, max_epochs)

In [19]:
wiki_vote_G_MIS = calculate_large_G(wiki_vote_G, 1000)

INFO:root:Total communities 32
INFO:root:Community 1 processing...
INFO:root:Build network on graph G:Graph with 957 nodes and 7504 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root:Epoch = 19
DEBUG:root:Epoch = 20
DEBUG:root:Epoch = 21
DEBUG:root:Epoch = 22
DEBUG:root:Epoch = 23
DEBUG:root:Epoch = 24
DEBUG:root:Epoch = 25
DEBUG:root:Epoch = 26
DEBUG:root:Epoch = 27
DEBUG:root:Epoch = 28
DEBUG:root:Epoch = 29
DEBUG:root:Epoch = 30
DEBUG:root:Epoch = 31
DEBUG:root:Epoch = 32
DEBUG:root:Epoch = 33
DEBUG:root:Epoch = 34
DEBUG:root:Epoch = 35
DEBUG:root:Epoch = 36
DEBUG:root:Epoch = 37
DEBUG:root:Epoch = 38
DEBUG:root:Ep

DEBUG:root:Epoch = 355
DEBUG:root:Epoch = 356
DEBUG:root:Epoch = 357
DEBUG:root:Epoch = 358
DEBUG:root:Epoch = 359
DEBUG:root:Epoch = 360
DEBUG:root:Epoch = 361
DEBUG:root:Epoch = 362
DEBUG:root:Epoch = 363
DEBUG:root:Epoch = 364
DEBUG:root:Epoch = 365
DEBUG:root:Epoch = 366
DEBUG:root:Epoch = 367
DEBUG:root:Epoch = 368
DEBUG:root:Epoch = 369
DEBUG:root:Epoch = 370
DEBUG:root:Epoch = 371
DEBUG:root:Epoch = 372
DEBUG:root:Epoch = 373
DEBUG:root:Epoch = 374
DEBUG:root:Epoch = 375
DEBUG:root:Epoch = 376
DEBUG:root:Epoch = 377
DEBUG:root:Epoch = 378
DEBUG:root:Epoch = 379
DEBUG:root:Epoch = 380
DEBUG:root:Epoch = 381
DEBUG:root:Epoch = 382
DEBUG:root:Epoch = 383
DEBUG:root:Epoch = 384
DEBUG:root:Epoch = 385
DEBUG:root:Epoch = 386
DEBUG:root:Epoch = 387
DEBUG:root:Epoch = 388
DEBUG:root:Epoch = 389
DEBUG:root:Epoch = 390
DEBUG:root:Epoch = 391
DEBUG:root:Epoch = 392
DEBUG:root:Epoch = 393
DEBUG:root:Epoch = 394
DEBUG:root:Epoch = 395
DEBUG:root:Epoch = 396
DEBUG:root:Epoch = 397
DEBUG:root:

DEBUG:root:Epoch = 712
DEBUG:root:Epoch = 713
DEBUG:root:Epoch = 714
DEBUG:root:Epoch = 715
DEBUG:root:Epoch = 716
DEBUG:root:Epoch = 717
DEBUG:root:Epoch = 718
DEBUG:root:Epoch = 719
DEBUG:root:Epoch = 720
DEBUG:root:Epoch = 721
DEBUG:root:Epoch = 722
DEBUG:root:Epoch = 723
DEBUG:root:Epoch = 724
DEBUG:root:Epoch = 725
DEBUG:root:Epoch = 726
DEBUG:root:Epoch = 727
DEBUG:root:Epoch = 728
DEBUG:root:Epoch = 729
DEBUG:root:Epoch = 730
DEBUG:root:Epoch = 731
DEBUG:root:Epoch = 732
DEBUG:root:Epoch = 733
DEBUG:root:Epoch = 734
DEBUG:root:Epoch = 735
DEBUG:root:Epoch = 736
DEBUG:root:Epoch = 737
DEBUG:root:Epoch = 738
DEBUG:root:Epoch = 739
DEBUG:root:Epoch = 740
DEBUG:root:Epoch = 741
DEBUG:root:Epoch = 742
DEBUG:root:Epoch = 743
DEBUG:root:Epoch = 744
DEBUG:root:Epoch = 745
DEBUG:root:Epoch = 746
DEBUG:root:Epoch = 747
DEBUG:root:Epoch = 748
DEBUG:root:Epoch = 749
DEBUG:root:Epoch = 750
DEBUG:root:Epoch = 751
DEBUG:root:Epoch = 752
DEBUG:root:Epoch = 753
DEBUG:root:Epoch = 754
DEBUG:root:

DEBUG:root:Epoch = 66
DEBUG:root:Epoch = 67
DEBUG:root:Epoch = 68
DEBUG:root:Epoch = 69
DEBUG:root:Epoch = 70
DEBUG:root:Epoch = 71
DEBUG:root:Epoch = 72
DEBUG:root:Epoch = 73
DEBUG:root:Epoch = 74
DEBUG:root:Epoch = 75
DEBUG:root:Epoch = 76
DEBUG:root:Epoch = 77
DEBUG:root:Epoch = 78
DEBUG:root:Epoch = 79
DEBUG:root:Epoch = 80
DEBUG:root:Epoch = 81
DEBUG:root:Epoch = 82
DEBUG:root:Epoch = 83
DEBUG:root:Epoch = 84
DEBUG:root:Epoch = 85
DEBUG:root:Epoch = 86
DEBUG:root:Epoch = 87
DEBUG:root:Epoch = 88
DEBUG:root:Epoch = 89
DEBUG:root:Epoch = 90
DEBUG:root:Epoch = 91
DEBUG:root:Epoch = 92
DEBUG:root:Epoch = 93
DEBUG:root:Epoch = 94
DEBUG:root:Epoch = 95
DEBUG:root:Epoch = 96
DEBUG:root:Epoch = 97
DEBUG:root:Epoch = 98
DEBUG:root:Epoch = 99
DEBUG:root:Epoch = 100
DEBUG:root:Epoch = 101
DEBUG:root:Epoch = 102
DEBUG:root:Epoch = 103
DEBUG:root:Epoch = 104
DEBUG:root:Epoch = 105
DEBUG:root:Epoch = 106
DEBUG:root:Epoch = 107
DEBUG:root:Epoch = 108
DEBUG:root:Epoch = 109
DEBUG:root:Epoch = 110

DEBUG:root:Epoch = 424
DEBUG:root:Epoch = 425
DEBUG:root:Epoch = 426
DEBUG:root:Epoch = 427
DEBUG:root:Epoch = 428
DEBUG:root:Epoch = 429
DEBUG:root:Epoch = 430
DEBUG:root:Epoch = 431
DEBUG:root:Epoch = 432
DEBUG:root:Epoch = 433
DEBUG:root:Epoch = 434
DEBUG:root:Epoch = 435
DEBUG:root:Epoch = 436
DEBUG:root:Epoch = 437
DEBUG:root:Epoch = 438
DEBUG:root:Epoch = 439
DEBUG:root:Epoch = 440
DEBUG:root:Epoch = 441
DEBUG:root:Epoch = 442
DEBUG:root:Epoch = 443
DEBUG:root:Epoch = 444
DEBUG:root:Epoch = 445
DEBUG:root:Epoch = 446
DEBUG:root:Epoch = 447
DEBUG:root:Epoch = 448
DEBUG:root:Epoch = 449
DEBUG:root:Epoch = 450
DEBUG:root:Epoch = 451
DEBUG:root:Epoch = 452
DEBUG:root:Epoch = 453
DEBUG:root:Epoch = 454
DEBUG:root:Epoch = 455
DEBUG:root:Epoch = 456
DEBUG:root:Epoch = 457
DEBUG:root:Epoch = 458
DEBUG:root:Epoch = 459
DEBUG:root:Epoch = 460
DEBUG:root:Epoch = 461
DEBUG:root:Epoch = 462
DEBUG:root:Epoch = 463
DEBUG:root:Epoch = 464
DEBUG:root:Epoch = 465
DEBUG:root:Epoch = 466
DEBUG:root:

DEBUG:root:Epoch = 781
DEBUG:root:Epoch = 782
DEBUG:root:Epoch = 783
DEBUG:root:Epoch = 784
DEBUG:root:Epoch = 785
DEBUG:root:Epoch = 786
DEBUG:root:Epoch = 787
DEBUG:root:Epoch = 788
DEBUG:root:Epoch = 789
DEBUG:root:Epoch = 790
DEBUG:root:Epoch = 791
DEBUG:root:Epoch = 792
DEBUG:root:Epoch = 793
DEBUG:root:Epoch = 794
DEBUG:root:Epoch = 795
DEBUG:root:Epoch = 796
DEBUG:root:Epoch = 797
DEBUG:root:Epoch = 798
DEBUG:root:Epoch = 799
DEBUG:root:Epoch = 800
DEBUG:root:Epoch = 801
DEBUG:root:Epoch = 802
DEBUG:root:Epoch = 803
DEBUG:root:Epoch = 804
DEBUG:root:Epoch = 805
DEBUG:root:Epoch = 806
DEBUG:root:Epoch = 807
DEBUG:root:Epoch = 808
DEBUG:root:Epoch = 809
DEBUG:root:Epoch = 810
DEBUG:root:Epoch = 811
DEBUG:root:Epoch = 812
DEBUG:root:Epoch = 813
DEBUG:root:Epoch = 814
DEBUG:root:Epoch = 815
DEBUG:root:Epoch = 816
DEBUG:root:Epoch = 817
DEBUG:root:Epoch = 818
DEBUG:root:Epoch = 819
DEBUG:root:Epoch = 820
DEBUG:root:Epoch = 821
DEBUG:root:Epoch = 822
DEBUG:root:Epoch = 823
DEBUG:root:

DEBUG:root:Epoch = 136
DEBUG:root:Epoch = 137
DEBUG:root:Epoch = 138
DEBUG:root:Epoch = 139
DEBUG:root:Epoch = 140
DEBUG:root:Epoch = 141
DEBUG:root:Epoch = 142
DEBUG:root:Epoch = 143
DEBUG:root:Epoch = 144
DEBUG:root:Epoch = 145
DEBUG:root:Epoch = 146
DEBUG:root:Epoch = 147
DEBUG:root:Epoch = 148
DEBUG:root:Epoch = 149
DEBUG:root:Epoch = 150
DEBUG:root:Epoch = 151
DEBUG:root:Epoch = 152
DEBUG:root:Epoch = 153
DEBUG:root:Epoch = 154
DEBUG:root:Epoch = 155
DEBUG:root:Epoch = 156
DEBUG:root:Epoch = 157
DEBUG:root:Epoch = 158
DEBUG:root:Epoch = 159
DEBUG:root:Epoch = 160
DEBUG:root:Epoch = 161
DEBUG:root:Epoch = 162
DEBUG:root:Epoch = 163
DEBUG:root:Epoch = 164
DEBUG:root:Epoch = 165
DEBUG:root:Epoch = 166
DEBUG:root:Epoch = 167
DEBUG:root:Epoch = 168
DEBUG:root:Epoch = 169
DEBUG:root:Epoch = 170
DEBUG:root:Epoch = 171
DEBUG:root:Epoch = 172
DEBUG:root:Epoch = 173
DEBUG:root:Epoch = 174
DEBUG:root:Epoch = 175
DEBUG:root:Epoch = 176
DEBUG:root:Epoch = 177
DEBUG:root:Epoch = 178
DEBUG:root:

DEBUG:root:Epoch = 493
DEBUG:root:Epoch = 494
DEBUG:root:Epoch = 495
DEBUG:root:Epoch = 496
DEBUG:root:Epoch = 497
DEBUG:root:Epoch = 498
DEBUG:root:Epoch = 499
DEBUG:root:Epoch = 500
DEBUG:root:Epoch = 501
DEBUG:root:Epoch = 502
DEBUG:root:Epoch = 503
DEBUG:root:Epoch = 504
DEBUG:root:Epoch = 505
DEBUG:root:Epoch = 506
DEBUG:root:Epoch = 507
DEBUG:root:Epoch = 508
DEBUG:root:Epoch = 509
DEBUG:root:Epoch = 510
DEBUG:root:Epoch = 511
DEBUG:root:Epoch = 512
DEBUG:root:Epoch = 513
DEBUG:root:Epoch = 514
DEBUG:root:Epoch = 515
DEBUG:root:Epoch = 516
DEBUG:root:Epoch = 517
DEBUG:root:Epoch = 518
DEBUG:root:Epoch = 519
DEBUG:root:Epoch = 520
DEBUG:root:Epoch = 521
DEBUG:root:Epoch = 522
DEBUG:root:Epoch = 523
DEBUG:root:Epoch = 524
DEBUG:root:Epoch = 525
DEBUG:root:Epoch = 526
DEBUG:root:Epoch = 527
DEBUG:root:Epoch = 528
DEBUG:root:Epoch = 529
DEBUG:root:Epoch = 530
DEBUG:root:Epoch = 531
DEBUG:root:Epoch = 532
DEBUG:root:Epoch = 533
DEBUG:root:Epoch = 534
DEBUG:root:Epoch = 535
DEBUG:root:

DEBUG:root:Epoch = 850
DEBUG:root:Epoch = 851
DEBUG:root:Epoch = 852
DEBUG:root:Epoch = 853
DEBUG:root:Epoch = 854
DEBUG:root:Epoch = 855
DEBUG:root:Epoch = 856
DEBUG:root:Epoch = 857
DEBUG:root:Epoch = 858
DEBUG:root:Epoch = 859
DEBUG:root:Epoch = 860
DEBUG:root:Epoch = 861
DEBUG:root:Epoch = 862
DEBUG:root:Epoch = 863
DEBUG:root:Epoch = 864
DEBUG:root:Epoch = 865
DEBUG:root:Epoch = 866
DEBUG:root:Epoch = 867
DEBUG:root:Epoch = 868
DEBUG:root:Epoch = 869
DEBUG:root:Epoch = 870
DEBUG:root:Epoch = 871
DEBUG:root:Epoch = 872
DEBUG:root:Epoch = 873
DEBUG:root:Epoch = 874
DEBUG:root:Epoch = 875
DEBUG:root:Epoch = 876
DEBUG:root:Epoch = 877
DEBUG:root:Epoch = 878
DEBUG:root:Epoch = 879
DEBUG:root:Epoch = 880
DEBUG:root:Epoch = 881
DEBUG:root:Epoch = 882
DEBUG:root:Epoch = 883
DEBUG:root:Epoch = 884
DEBUG:root:Epoch = 885
DEBUG:root:Epoch = 886
DEBUG:root:Epoch = 887
DEBUG:root:Epoch = 888
DEBUG:root:Epoch = 889
DEBUG:root:Epoch = 890
DEBUG:root:Epoch = 891
DEBUG:root:Epoch = 892
DEBUG:root:

KeyboardInterrupt: 

In [77]:
def test_average_result_on_graph(G, graph_name, iterations = 10):
    sum = 0
    for i in range(iterations):
        er1 = calculate_large_G(G, 50)
        sum+=len(er1)
    print("dNN result on {} = {}".format(graph_name,sum/iterations))

In [35]:
test_average_result_on_graph(nx.erdos_renyi_graph(100,0.1),"ER 100, 0.1")

INFO:root:Total communities 9
INFO:root:Community 1 processing...
INFO:root:Build network on graph G:Graph with 14 nodes and 31 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root:Epoch = 19
DEBUG:root:Epoch = 20
DEBUG:root:Epoch = 21
DEBUG:root:Epoch = 22
DEBUG:root:Epoch = 23
DEBUG:root:Epoch = 24
DEBUG:root:Epoch = 25
DEBUG:root:Epoch = 26
DEBUG:root:Epoch = 27
DEBUG:root:Epoch = 28
DEBUG:root:Epoch = 29
DEBUG:root:Optimal solution found. Cardinality = 5
DEBUG:root:Total epochs = 30
INFO:root:Community 2 processing...
INFO:root:Build network on graph G:Graph with 10 nodes and 18 edges.
DEBUG:root:Epoch = 0
DEBUG:roo

DEBUG:root:Epoch = 27
DEBUG:root:Epoch = 28
DEBUG:root:Epoch = 29
DEBUG:root:Epoch = 30
DEBUG:root:Epoch = 31
DEBUG:root:Epoch = 32
DEBUG:root:Epoch = 33
DEBUG:root:Epoch = 34
DEBUG:root:Epoch = 35
DEBUG:root:Epoch = 36
DEBUG:root:Epoch = 37
DEBUG:root:Epoch = 38
DEBUG:root:Epoch = 39
DEBUG:root:Epoch = 40
DEBUG:root:Epoch = 41
DEBUG:root:Epoch = 42
DEBUG:root:Epoch = 43
DEBUG:root:Epoch = 44
DEBUG:root:Epoch = 45
DEBUG:root:Epoch = 46
DEBUG:root:Epoch = 47
DEBUG:root:Epoch = 48
DEBUG:root:Epoch = 49
DEBUG:root:Total epochs = 50
INFO:root:Community 2 processing...
INFO:root:Build network on graph G:Graph with 15 nodes and 30 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Optimal solution found. Cardinality = 5
DEBUG:root:Total epochs = 2
INFO:root:Community 3 processing...
INFO:root:Build network on graph G:Graph with 8 nodes and 17 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 4 processing...
INFO:

DEBUG:root:Epoch = 43
DEBUG:root:Epoch = 44
DEBUG:root:Epoch = 45
DEBUG:root:Epoch = 46
DEBUG:root:Epoch = 47
DEBUG:root:Epoch = 48
DEBUG:root:Epoch = 49
DEBUG:root:Total epochs = 50
INFO:root:Community 4 processing...
INFO:root:Build network on graph G:Graph with 11 nodes and 26 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root:Epoch = 19
DEBUG:root:Epoch = 20
DEBUG:root:Epoch = 21
DEBUG:root:Epoch = 22
DEBUG:root:Epoch = 23
DEBUG:root:Epoch = 24
DEBUG:root:Optimal solution found. Cardinality = 4
DEBUG:root:Total epochs = 25
INFO:root:Community 5 processing...
INFO:root:Build network on graph G:Graph with 8 nodes an

INFO:root:Community 6 processing...
INFO:root:Build network on graph G:Graph with 9 nodes and 13 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root:Epoch = 19
DEBUG:root:Epoch = 20
DEBUG:root:Epoch = 21
DEBUG:root:Epoch = 22
DEBUG:root:Epoch = 23
DEBUG:root:Epoch = 24
DEBUG:root:Epoch = 25
DEBUG:root:Epoch = 26
DEBUG:root:Epoch = 27
DEBUG:root:Epoch = 28
DEBUG:root:Epoch = 29
DEBUG:root:Epoch = 30
DEBUG:root:Epoch = 31
DEBUG:root:Epoch = 32
DEBUG:root:Epoch = 33
DEBUG:root:Epoch = 34
DEBUG:root:Epoch = 35
DEBUG:root:Epoch = 36
DEBUG:root:Epoch = 37
DEBUG:root:Epoch = 38
DEBUG:root:Epoch = 39
DEBUG:root:Epoch = 40
DEBU

DEBUG:root:Epoch = 28
DEBUG:root:Epoch = 29
DEBUG:root:Epoch = 30
DEBUG:root:Epoch = 31
DEBUG:root:Epoch = 32
DEBUG:root:Epoch = 33
DEBUG:root:Epoch = 34
DEBUG:root:Epoch = 35
DEBUG:root:Epoch = 36
DEBUG:root:Epoch = 37
DEBUG:root:Epoch = 38
DEBUG:root:Epoch = 39
DEBUG:root:Epoch = 40
DEBUG:root:Epoch = 41
DEBUG:root:Epoch = 42
DEBUG:root:Epoch = 43
DEBUG:root:Epoch = 44
DEBUG:root:Epoch = 45
DEBUG:root:Epoch = 46
DEBUG:root:Epoch = 47
DEBUG:root:Epoch = 48
DEBUG:root:Epoch = 49
DEBUG:root:Total epochs = 50
INFO:root:Community 8 processing...
INFO:root:Build network on graph G:Graph with 4 nodes and 4 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 1

DEBUG:root:Epoch = 28
DEBUG:root:Epoch = 29
DEBUG:root:Epoch = 30
DEBUG:root:Epoch = 31
DEBUG:root:Epoch = 32
DEBUG:root:Epoch = 33
DEBUG:root:Epoch = 34
DEBUG:root:Epoch = 35
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 36
INFO:root:Community 4 processing...
INFO:root:Build network on graph G:Graph with 10 nodes and 23 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root:Epoch = 19
DEBUG:root:Epoch = 20
DEBUG:root:Epoch = 21
DEBUG:root:Epoch = 22
DEBUG:root:Epoch = 23
DEBUG:root:Epoch = 24
DEBUG:root:Epoch = 25
DEBUG:root:Epoch = 26
DEBUG:root:Epoch = 27
DEBUG:root:Epoch = 28
DEBUG:root:Epoch = 29
DEBUG:root:Epoch

DEBUG:root:Epoch = 23
DEBUG:root:Epoch = 24
DEBUG:root:Epoch = 25
DEBUG:root:Epoch = 26
DEBUG:root:Epoch = 27
DEBUG:root:Epoch = 28
DEBUG:root:Epoch = 29
DEBUG:root:Epoch = 30
DEBUG:root:Epoch = 31
DEBUG:root:Epoch = 32
DEBUG:root:Epoch = 33
DEBUG:root:Epoch = 34
DEBUG:root:Epoch = 35
DEBUG:root:Epoch = 36
DEBUG:root:Epoch = 37
DEBUG:root:Epoch = 38
DEBUG:root:Epoch = 39
DEBUG:root:Epoch = 40
DEBUG:root:Epoch = 41
DEBUG:root:Epoch = 42
DEBUG:root:Epoch = 43
DEBUG:root:Epoch = 44
DEBUG:root:Epoch = 45
DEBUG:root:Epoch = 46
DEBUG:root:Epoch = 47
DEBUG:root:Epoch = 48
DEBUG:root:Epoch = 49
DEBUG:root:Total epochs = 50
INFO:root:Community 8 processing...
INFO:root:Build network on graph G:Graph with 9 nodes and 15 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Optimal solution found. Cardinality = 4
DEBUG:root:Total epochs = 6
INFO:root:Community 9 processing...
INFO:root:Build network on graph

DEBUG:root:Length of F = 104
DEBUG:root:Length of F = 96
DEBUG:root:Length of F = 88
DEBUG:root:Length of F = 72
DEBUG:root:Length of F = 68
DEBUG:root:Length of F = 56
DEBUG:root:Length of F = 40
DEBUG:root:Length of F = 36
DEBUG:root:Length of F = 32
DEBUG:root:Length of F = 28
DEBUG:root:Length of F = 24
DEBUG:root:Length of F = 20
DEBUG:root:Length of F = 16
DEBUG:root:Length of F = 8
DEBUG:root:Length of F = 4
INFO:root:Left nodes size = 13
INFO:root:Build network on graph G:Graph with 13 nodes and 7 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Optimal solution found. Cardinality = 9
DEBUG:root:Total epochs = 2
INFO:root:Total communities 9
INFO:root:Community 1 processing...
INFO:root:Build network on graph G:Graph with 14 nodes and 34 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch

dNN result on ER 100, 0.1 = 26.5


In [36]:
test_average_result_on_graph(nx.erdos_renyi_graph(100,0.2),"ER 100, 0.2")

INFO:root:Total communities 11
INFO:root:Community 1 processing...
INFO:root:Build network on graph G:Graph with 7 nodes and 14 edges.
DEBUG:root:Optimal solution found. Cardinality = 3
DEBUG:root:Total epochs = 0
INFO:root:Community 2 processing...
INFO:root:Build network on graph G:Graph with 15 nodes and 45 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root:Epoch = 19
DEBUG:root:Epoch = 20
DEBUG:root:Epoch = 21
DEBUG:root:Epoch = 22
DEBUG:root:Epoch = 23
DEBUG:root:Epoch = 24
DEBUG:root:Epoch = 25
DEBUG:root:Epoch = 26
DEBUG:root:Epoch = 27
DEBUG:root:Epoch = 28
DEBUG:root:Epoch = 29
DEBUG:root:Epoch = 30
DEBUG:roo

DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Optimal solution found. Cardinality = 3
DEBUG:root:Total epochs = 9
INFO:root:Community 6 processing...
INFO:root:Build network on graph G:Graph with 14 nodes and 41 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root:Epoch = 19
DEBUG:root:Epoch = 20
DEBUG:root:Epoch = 21
DEBUG:root:Epoch = 22
DEBUG:root:Epoch = 23
DEBUG:root:Epoch = 24
DEBUG:root:Epoch = 25
DEBUG:root:Optimal solution found. Cardinality = 4
DEBUG:root:Total epochs = 26
INFO:root:Community 7 processing...
INFO:root:Bu

DEBUG:root:Length of F = 32
DEBUG:root:Length of F = 28
DEBUG:root:Length of F = 24
DEBUG:root:Length of F = 12
DEBUG:root:Length of F = 4
INFO:root:Left nodes size = 34
INFO:root:Build network on graph G:Graph with 34 nodes and 95 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root:Epoch = 19
DEBUG:root:Epoch = 20
DEBUG:root:Epoch = 21
DEBUG:root:Epoch = 22
DEBUG:root:Epoch = 23
DEBUG:root:Epoch = 24
DEBUG:root:Epoch = 25
DEBUG:root:Epoch = 26
DEBUG:root:Epoch = 27
DEBUG:root:Epoch = 28
DEBUG:root:Epoch = 29
DEBUG:root:Epoch = 30
DEBUG:root:Epoch = 31
DEBUG:root:Epoch = 32
DEBUG:root:Epoch = 33
DEBUG:root:Epoch = 34
D

DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root:Epoch = 19
DEBUG:root:Epoch = 20
DEBUG:root:Epoch = 21
DEBUG:root:Epoch = 22
DEBUG:root:Epoch = 23
DEBUG:root:Epoch = 24
DEBUG:root:Epoch = 25
DEBUG:root:Epoch = 26
DEBUG:root:Epoch = 27
DEBUG:root:Epoch = 28
DEBUG:root:Epoch = 29
DEBUG:root:Epoch = 30
DEBUG:root:Epoch = 31
DEBUG:root:Epoch = 32
DEBUG:root:Epoch = 33
DEBUG:root:Epoch = 34
DEBUG:root:Epoch = 35
DEBUG:root:Epoch = 36
DEBUG:root:Epoch = 37
DEBUG:root:Epoch = 38
DEBUG:root:Epoch = 39
DEBUG:root:Epoch = 40
DEBUG:root:Epoch = 41
DEBUG:root:Epoch = 42
DEBUG:root:Epoch = 43
DEBUG:root:Epoch = 44
DEBUG:root:Epoch = 45
DEBUG:root:Epoch = 46
DEBUG:root:Epoch = 47
DEBUG:root:Epoch = 48
DEBUG:root:Epoch = 49
DEBUG:root:Total epochs = 50
INFO:root:Community 3 processing...
INFO:root:Bui

DEBUG:root:Epoch = 27
DEBUG:root:Epoch = 28
DEBUG:root:Epoch = 29
DEBUG:root:Epoch = 30
DEBUG:root:Epoch = 31
DEBUG:root:Epoch = 32
DEBUG:root:Epoch = 33
DEBUG:root:Epoch = 34
DEBUG:root:Epoch = 35
DEBUG:root:Epoch = 36
DEBUG:root:Epoch = 37
DEBUG:root:Epoch = 38
DEBUG:root:Epoch = 39
DEBUG:root:Epoch = 40
DEBUG:root:Epoch = 41
DEBUG:root:Epoch = 42
DEBUG:root:Epoch = 43
DEBUG:root:Epoch = 44
DEBUG:root:Epoch = 45
DEBUG:root:Epoch = 46
DEBUG:root:Epoch = 47
DEBUG:root:Epoch = 48
DEBUG:root:Epoch = 49
DEBUG:root:Total epochs = 50
INFO:root:Community 8 processing...
INFO:root:Build network on graph G:Graph with 6 nodes and 11 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 9 processing...
INFO:root:Build network on graph G:Graph with 12 nodes and 32 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 10 processing...
INFO:root:Build network on grap

INFO:root:Left nodes size = 18
INFO:root:Build network on graph G:Graph with 18 nodes and 34 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Optimal solution found. Cardinality = 8
DEBUG:root:Total epochs = 3
INFO:root:Total communities 12
INFO:root:Community 1 processing...
INFO:root:Build network on graph G:Graph with 8 nodes and 17 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 2 processing...
INFO:root:Build network on graph G:Graph with 5 nodes and 7 edges.
DEBUG:root:Optimal solution found. Cardinality = 2
DEBUG:root:Total epochs = 0
INFO:root:Community 3 processing...
INFO:root:Build network on graph G:Graph with 11 nodes and 30 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 4 processing...
INFO:root:Build network on graph G:Graph with 9 nodes and 22 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Optimal solu

DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 5 processing...
INFO:root:Build network on graph G:Graph with 10 nodes and 25 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 18
INFO:root:Community 6 processing...
INFO:root:Build network on graph G:Graph with 7 nodes and 13 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 7 processing...
INFO:root:Build network on graph G:Graph with 19 nodes and 65 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch 

DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Replacing forbiden nodes
DEBUG:root:Length of F = 36
DEBUG:root:Length of F = 32
DEBUG:root:Length of F = 24
DEBUG:root:Length of F = 16
DEBUG:root:Length of F = 12
INFO:root:Left nodes size = 21
INFO:root:Build network on graph G:Graph with 21 nodes and 48 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root:Epoch = 19
DEBUG:root:Epoch = 20
DEBUG:root:Epoch = 21
DEBUG:root:Epoch = 22
DEBUG:root:Epoch = 23
DEBUG:root:Epoch = 24
DEBUG:root:Epoch = 25
DEBUG:root:Epoch = 26
DEBUG:root:Epoch = 27
DEBUG:root:Epoch = 28
DEB

dNN result on ER 100, 0.2 = 17.1


In [37]:
test_average_result_on_graph(nx.erdos_renyi_graph(200,0.1),"ER 200, 0.1")

INFO:root:Total communities 14
INFO:root:Community 1 processing...
INFO:root:Build network on graph G:Graph with 18 nodes and 46 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root:Epoch = 19
DEBUG:root:Epoch = 20
DEBUG:root:Epoch = 21
DEBUG:root:Epoch = 22
DEBUG:root:Epoch = 23
DEBUG:root:Epoch = 24
DEBUG:root:Epoch = 25
DEBUG:root:Epoch = 26
DEBUG:root:Epoch = 27
DEBUG:root:Epoch = 28
DEBUG:root:Epoch = 29
DEBUG:root:Epoch = 30
DEBUG:root:Epoch = 31
DEBUG:root:Epoch = 32
DEBUG:root:Epoch = 33
DEBUG:root:Epoch = 34
DEBUG:root:Epoch = 35
DEBUG:root:Epoch = 36
DEBUG:root:Epoch = 37
DEBUG:root:Epoch = 38
DEBUG:root:Epoch

DEBUG:root:Length of F = 196
DEBUG:root:Length of F = 180
DEBUG:root:Length of F = 160
DEBUG:root:Length of F = 148
DEBUG:root:Length of F = 144
DEBUG:root:Length of F = 140
DEBUG:root:Length of F = 128
DEBUG:root:Length of F = 124
DEBUG:root:Length of F = 108
DEBUG:root:Length of F = 96
DEBUG:root:Length of F = 88
DEBUG:root:Length of F = 76
DEBUG:root:Length of F = 60
DEBUG:root:Length of F = 56
DEBUG:root:Length of F = 48
DEBUG:root:Length of F = 44
DEBUG:root:Length of F = 36
DEBUG:root:Length of F = 32
DEBUG:root:Length of F = 28
DEBUG:root:Length of F = 20
DEBUG:root:Length of F = 16
DEBUG:root:Length of F = 12
DEBUG:root:Length of F = 8
DEBUG:root:Length of F = 4
INFO:root:Left nodes size = 53
INFO:root:Build network on graph G:Graph with 53 nodes and 133 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root

DEBUG:root:Epoch = 26
DEBUG:root:Epoch = 27
DEBUG:root:Epoch = 28
DEBUG:root:Epoch = 29
DEBUG:root:Epoch = 30
DEBUG:root:Epoch = 31
DEBUG:root:Epoch = 32
DEBUG:root:Epoch = 33
DEBUG:root:Epoch = 34
DEBUG:root:Epoch = 35
DEBUG:root:Epoch = 36
DEBUG:root:Epoch = 37
DEBUG:root:Epoch = 38
DEBUG:root:Epoch = 39
DEBUG:root:Epoch = 40
DEBUG:root:Epoch = 41
DEBUG:root:Epoch = 42
DEBUG:root:Epoch = 43
DEBUG:root:Epoch = 44
DEBUG:root:Optimal solution found. Cardinality = 6
DEBUG:root:Total epochs = 45
INFO:root:Community 11 processing...
INFO:root:Build network on graph G:Graph with 15 nodes and 33 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root

DEBUG:root:Epoch = 44
DEBUG:root:Epoch = 45
DEBUG:root:Epoch = 46
DEBUG:root:Epoch = 47
DEBUG:root:Epoch = 48
DEBUG:root:Epoch = 49
DEBUG:root:Total epochs = 50
INFO:root:Community 5 processing...
INFO:root:Build network on graph G:Graph with 22 nodes and 68 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root:Epoch = 19
DEBUG:root:Epoch = 20
DEBUG:root:Epoch = 21
DEBUG:root:Epoch = 22
DEBUG:root:Epoch = 23
DEBUG:root:Epoch = 24
DEBUG:root:Epoch = 25
DEBUG:root:Epoch = 26
DEBUG:root:Epoch = 27
DEBUG:root:Epoch = 28
DEBUG:root:Epoch = 29
DEBUG:root:Epoch = 30
DEBUG:root:Epoch = 31
DEBUG:root:Epoch = 32
DEBUG:root:Epoch =

DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root:Epoch = 19
DEBUG:root:Epoch = 20
DEBUG:root:Epoch = 21
DEBUG:root:Epoch = 22
DEBUG:root:Epoch = 23
DEBUG:root:Epoch = 24
DEBUG:root:Epoch = 25
DEBUG:root:Epoch = 26
DEBUG:root:Epoch = 27
DEBUG:root:Epoch = 28
DEBUG:root:Epoch = 29
DEBUG:root:Epoch = 30
DEBUG:root:Epoch = 31
DEBUG:root:Epoch = 32
DEBUG:root:Epoch = 33
DEBUG:root:Epoch = 34
DEBUG:root:Epoch = 35
DEBUG:root:Epoch = 36
DEBUG:root:Epoch = 37
DEBUG:root:Epoch = 38
DEBUG:root:Epoch = 39
DEBUG:root:Epoch = 40
DEBUG:root:Epoch = 41
DEBUG:root:Optimal solution found. Cardinality = 21
DEBUG:root:Total epochs = 42
INFO:root:Total communities 14
INFO:root:Community 1 processing...
INFO:root:Build network on graph G:Graph with 11 nodes and 24 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch =

...

DEBUG:root:Epoch = 36
DEBUG:root:Epoch = 37
DEBUG:root:Epoch = 38
DEBUG:root:Epoch = 39
DEBUG:root:Epoch = 40
DEBUG:root:Epoch = 41
DEBUG:root:Epoch = 42
DEBUG:root:Epoch = 43
DEBUG:root:Optimal solution found. Cardinality = 3
DEBUG:root:Total epochs = 44
INFO:root:Community 2 processing...
INFO:root:Build network on graph G:Graph with 14 nodes and 32 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root:Epoch = 19
DEBUG:root:Epoch = 20
DEBUG:root:Epoch = 21
DEBUG:root:Epoch = 22
DEBUG:root:Epoch = 23
DEBUG:root:Epoch = 24
DEBUG:root:Epoch = 25
DEBUG:root:Epoch = 26
DEBUG:root:Epoch = 27
DEBUG:root:Epoch = 28
DEBUG:root:

DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root:Epoch = 19
DEBUG:root:Epoch = 20
DEBUG:root:Epoch = 21
DEBUG:root:Epoch = 22
DEBUG:root:Epoch = 23
DEBUG:root:Epoch = 24
DEBUG:root:Epoch = 25
DEBUG:root:Epoch = 26
DEBUG:root:Epoch = 27
DEBUG:root:Epoch = 28
DEBUG:root:Epoch = 29
DEBUG:root:Epoch = 30
DEBUG:root:Epoch = 31
DEBUG:root:Epoch = 32
DEBUG:root:Epoch = 33
DEBUG:root:Epoch = 34
DEBUG:root:Epoch = 35
DEBUG:root:Epoch = 36
DEBUG:root:Epoch = 37
DEBUG:root:Epoch = 38
DEBUG:root:Epoch = 39
DEBUG:root:Epoch = 40
DEBUG:root:Epoch = 41
DEBUG:root:Epoch = 42
DEBUG:root:Epoch = 43
DEBUG:root:Epoch = 44
DEBUG:root:Epoch = 4

INFO:root:Build network on graph G:Graph with 19 nodes and 55 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root:Epoch = 19
DEBUG:root:Epoch = 20
DEBUG:root:Epoch = 21
DEBUG:root:Epoch = 22
DEBUG:root:Epoch = 23
DEBUG:root:Epoch = 24
DEBUG:root:Epoch = 25
DEBUG:root:Epoch = 26
DEBUG:root:Epoch = 27
DEBUG:root:Epoch = 28
DEBUG:root:Epoch = 29
DEBUG:root:Epoch = 30
DEBUG:root:Epoch = 31
DEBUG:root:Epoch = 32
DEBUG:root:Epoch = 33
DEBUG:root:Epoch = 34
DEBUG:root:Epoch = 35
DEBUG:root:Epoch = 36
DEBUG:root:Epoch = 37
DEBUG:root:Epoch = 38
DEBUG:root:Epoch = 39
DEBUG:root:Epoch = 40
DEBUG:root:Epoch = 41
DEBUG:root:Epoch 

DEBUG:root:Length of F = 408
DEBUG:root:Length of F = 400
DEBUG:root:Length of F = 392
DEBUG:root:Length of F = 384
DEBUG:root:Length of F = 356
DEBUG:root:Length of F = 344
DEBUG:root:Length of F = 316
DEBUG:root:Length of F = 300
DEBUG:root:Length of F = 284
DEBUG:root:Length of F = 260
DEBUG:root:Length of F = 256
DEBUG:root:Length of F = 236
DEBUG:root:Length of F = 232
DEBUG:root:Length of F = 220
DEBUG:root:Length of F = 204
DEBUG:root:Length of F = 192
DEBUG:root:Length of F = 184
DEBUG:root:Length of F = 180
DEBUG:root:Length of F = 176
DEBUG:root:Length of F = 160
DEBUG:root:Length of F = 148
DEBUG:root:Length of F = 136
DEBUG:root:Length of F = 124
DEBUG:root:Length of F = 112
DEBUG:root:Length of F = 104
DEBUG:root:Length of F = 96
DEBUG:root:Length of F = 92
DEBUG:root:Length of F = 80
DEBUG:root:Length of F = 76
DEBUG:root:Length of F = 72
DEBUG:root:Length of F = 64
DEBUG:root:Length of F = 52
DEBUG:root:Length of F = 48
DEBUG:root:Length of F = 44
DEBUG:root:Length of F 

dNN result on ER 200, 0.1 = 26.2


In [38]:
test_average_result_on_graph(nx.erdos_renyi_graph(200,0.2),"ER 200, 0.2")

INFO:root:Total communities 15
INFO:root:Community 1 processing...
INFO:root:Build network on graph G:Graph with 13 nodes and 47 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 2 processing...
INFO:root:Build network on graph G:Graph with 10 nodes and 26 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 3 processing...
INFO:root:Build network on graph G:Graph with 12 nodes and 37 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Optimal solution found. Cardinality = 4
DEBUG:root:Total epochs = 1
INFO:root:Community 4 processing...
INFO:root:Build network on graph G:Graph with 10 nodes and 28 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:r

DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root:Epoch = 19
DEBUG:root:Epoch = 20
DEBUG:root:Epoch = 21
DEBUG:root:Epoch = 22
DEBUG:root:Epoch = 23
DEBUG:root:Epoch = 24
DEBUG:root:Epoch = 25
DEBUG:root:Epoch = 26
DEBUG:root:Epoch = 27
DEBUG:root:Epoch = 28
DEBUG:root:Epoch = 29
DEBUG:root:Epoch = 30
DEBUG:root:Epoch = 31
DEBUG:root:Epoch = 32
DEBUG:root:Epoch = 33
DEBUG:root:Epoch = 34
DEBUG:root:Epoch = 35
DEBUG:root:Epoch = 36
DEBUG:root:Epoch = 37
DEBUG:root:Epoch = 38
DEBUG:root:Epoch = 39
DEBUG:root:Epoch = 40
DEBUG:root:Epoch = 41
DEBUG:root:Epoch = 42
DEBUG:root:Epoch = 43
DEBUG:root:Epoch = 44
DEBUG:root:Epoch = 45
DEBUG:root:Epoch = 46
DEBUG:root:Epoch = 47
DEBUG:root:Epoch = 48
DEBUG:root:Epoch = 49
DEBUG:root:Total epochs = 50
INFO:root:Total communities 15
INFO:root:Community 1 processing...
INFO:root:Build network 

DEBUG:root:Length of F = 40
DEBUG:root:Length of F = 36
DEBUG:root:Length of F = 32
DEBUG:root:Length of F = 28
DEBUG:root:Length of F = 24
DEBUG:root:Length of F = 20
DEBUG:root:Length of F = 16
DEBUG:root:Length of F = 12
DEBUG:root:Length of F = 4
INFO:root:Left nodes size = 98
INFO:root:Build network on graph G:Graph with 98 nodes and 958 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root:Epoch = 19
DEBUG:root:Epoch = 20
DEBUG:root:Epoch = 21
DEBUG:root:Epoch = 22
DEBUG:root:Epoch = 23
DEBUG:root:Epoch = 24
DEBUG:root:Epoch = 25
DEBUG:root:Epoch = 26
DEBUG:root:Epoch = 27
DEBUG:root:Epoch = 28
DEBUG:root:Epoch = 2

DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Optimal solution found. Cardinality = 4
DEBUG:root:Total epochs = 17
INFO:root:Replacing forbiden nodes
DEBUG:root:Length of F = 140
DEBUG:root:Length of F = 124
DEBUG:root:Length of F = 112
DEBUG:root:Length of F = 96
DEBUG:root:Length of F = 92
DEBUG:root:Length of F = 80
DEBUG:root:Length of F = 68
DEBUG:root:Length of F = 60
DEBUG:root:Length of F = 48
DEBUG:root:Length of F = 40
DEBUG:root:Length of F = 28
DEBUG:root:Length of F = 20
DEBUG:root:Length of F = 16
DEBUG:root:Length of F = 8
DEBUG:root:Length of F = 4
INFO:root:Left nodes size = 46
INFO:root:Build network on graph G:Graph with 46 nodes and 188 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10

DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Replacing forbiden nodes
DEBUG:root:Length of F = 28
DEBUG:root:Length of F = 24
DEBUG:root:Length of F = 20
DEBUG:root:Length of F = 8
DEBUG:root:Length of F = 4
INFO:root:Left nodes size = 47
INFO:root:Build network on graph G:Graph with 47 nodes and 224 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root:Epoch = 19
DEBUG:root:Epoch = 20
DEBUG:root:Epoch = 21
DEBUG:root:Epoch = 22
DEBUG:root:Epoch = 23
DEBUG:root:Epoch = 24
DEBUG:root:Epoch = 25
DEBUG:root:Epoch = 26
DEBUG:root:Epoch = 27
DEBUG:root:Epoch = 28
DEBU

...

DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root:Epoch = 19
DEBUG:root:Epoch = 20
DEBUG:root:Epoch = 21
DEBUG:root:Epoch = 22
DEBUG:root:Epoch = 23
DEBUG:root:Epoch = 24
DEBUG:root:Epoch = 25
DEBUG:root:Epoch = 26
DEBUG:root:Epoch = 27
DEBUG:root:Epoch = 28
DEBUG:root:Epoch = 29
DEBUG:root:Epoch = 30
DEBUG:root:Epoch = 31
DEBUG:root:Epoch = 32
DEBUG:root:Epoch = 33
DEBUG:root:Epoch = 34
DEBUG:root:Epoch = 35
DEBUG:root:Epoch = 36
DEBUG:root:Epoch = 37
DEBUG:root:Epoch = 38
DEBUG:root:Epoch = 39
DEBUG:root:Epoch = 40
DEBUG:root:Epoch = 41
DEBUG:root:Epoch = 42
DEBUG:root:Epoch = 43
DEBUG:root:Epoch = 44
DEBUG:root:Epoch = 4

DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root:Epoch = 19
DEBUG:root:Epoch = 20
DEBUG:root:Epoch = 21
DEBUG:root:Epoch = 22
DEBUG:root:Epoch = 23
DEBUG:root:Epoch = 24
DEBUG:root:Epoch = 25
DEBUG:root:Epoch = 26
DEBUG:root:Epoch = 27
DEBUG:root:Epoch = 28
DEBUG:root:Epoch = 29
DEBUG:root:Epoch = 30
DEBUG:root:Epoch = 31
DEBUG:root:Epoch = 32
DEBUG:root:Epoch = 33
DEBUG:root:Epoch = 34
DEBUG:root:Optimal solution found. Cardinality = 3
DEBUG:root:Total epochs = 35
INFO:root:Community 12 processing...
INFO:root:Build network on graph G:Graph with 18 nodes and 66 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:E

DEBUG:root:Epoch = 20
DEBUG:root:Epoch = 21
DEBUG:root:Epoch = 22
DEBUG:root:Epoch = 23
DEBUG:root:Epoch = 24
DEBUG:root:Optimal solution found. Cardinality = 6
DEBUG:root:Total epochs = 25
INFO:root:Community 7 processing...
INFO:root:Build network on graph G:Graph with 15 nodes and 54 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 8 processing...
INFO:root:Build network on graph G:Graph with 17 nodes and 70 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root:Epoch = 19
DEBUG:root:Epoch = 20
DEBUG:root:Epoch = 21
DEBUG:root:Epoch = 22
DEBUG:root

DEBUG:root:Epoch = 19
DEBUG:root:Epoch = 20
DEBUG:root:Epoch = 21
DEBUG:root:Epoch = 22
DEBUG:root:Epoch = 23
DEBUG:root:Epoch = 24
DEBUG:root:Epoch = 25
DEBUG:root:Epoch = 26
DEBUG:root:Epoch = 27
DEBUG:root:Epoch = 28
DEBUG:root:Epoch = 29
DEBUG:root:Epoch = 30
DEBUG:root:Epoch = 31
DEBUG:root:Epoch = 32
DEBUG:root:Epoch = 33
DEBUG:root:Epoch = 34
DEBUG:root:Epoch = 35
DEBUG:root:Epoch = 36
DEBUG:root:Epoch = 37
DEBUG:root:Epoch = 38
DEBUG:root:Epoch = 39
DEBUG:root:Epoch = 40
DEBUG:root:Epoch = 41
DEBUG:root:Epoch = 42
DEBUG:root:Epoch = 43
DEBUG:root:Epoch = 44
DEBUG:root:Epoch = 45
DEBUG:root:Epoch = 46
DEBUG:root:Epoch = 47
DEBUG:root:Epoch = 48
DEBUG:root:Epoch = 49
DEBUG:root:Total epochs = 50
INFO:root:Community 7 processing...
INFO:root:Build network on graph G:Graph with 16 nodes and 55 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 8 processing...
INFO:root:Build network on graph G:Graph with 17 nodes

dNN result on ER 200, 0.2 = 11.7


In [42]:
test_average_result_on_graph(nx.stochastic_block_model([250],[[0.1]]),"SBM 250, 0.1")

INFO:root:Total communities 14
INFO:root:Community 1 processing...
INFO:root:Build network on graph G:Graph with 11 nodes and 26 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 2 processing...
INFO:root:Build network on graph G:Graph with 22 nodes and 72 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Optimal solution found. Cardinality = 7
DEBUG:root:Total epochs = 13
INFO:root:Community 3 processing...
INFO:root:Build network on graph G:Graph with 19 nodes and 52 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoc

DEBUG:root:Total epochs = 18
INFO:root:Community 10 processing...
INFO:root:Build network on graph G:Graph with 24 nodes and 72 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root:Epoch = 19
DEBUG:root:Epoch = 20
DEBUG:root:Optimal solution found. Cardinality = 6
DEBUG:root:Total epochs = 21
INFO:root:Community 11 processing...
INFO:root:Build network on graph G:Graph with 13 nodes and 34 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch 

DEBUG:root:Epoch = 41
DEBUG:root:Epoch = 42
DEBUG:root:Epoch = 43
DEBUG:root:Epoch = 44
DEBUG:root:Epoch = 45
DEBUG:root:Epoch = 46
DEBUG:root:Epoch = 47
DEBUG:root:Epoch = 48
DEBUG:root:Epoch = 49
DEBUG:root:Total epochs = 50
INFO:root:Community 4 processing...
INFO:root:Build network on graph G:Graph with 15 nodes and 39 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Optimal solution found. Cardinality = 5
DEBUG:root:Total epochs = 1
INFO:root:Community 5 processing...
INFO:root:Build network on graph G:Graph with 12 nodes and 30 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Optimal solution found. Cardinality = 5
DEBUG:root:Total epochs = 9
INFO:root:Community 6 processing...
INFO:root:Build network on graph G:Graph with 15 nodes and 41 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:

DEBUG:root:Length of F = 464
DEBUG:root:Length of F = 444
DEBUG:root:Length of F = 420
DEBUG:root:Length of F = 408
DEBUG:root:Length of F = 392
DEBUG:root:Length of F = 388
DEBUG:root:Length of F = 372
DEBUG:root:Length of F = 364
DEBUG:root:Length of F = 356
DEBUG:root:Length of F = 348
DEBUG:root:Length of F = 328
DEBUG:root:Length of F = 316
DEBUG:root:Length of F = 300
DEBUG:root:Length of F = 288
DEBUG:root:Length of F = 276
DEBUG:root:Length of F = 272
DEBUG:root:Length of F = 260
DEBUG:root:Length of F = 248
DEBUG:root:Length of F = 232
DEBUG:root:Length of F = 220
DEBUG:root:Length of F = 208
DEBUG:root:Length of F = 200
DEBUG:root:Length of F = 188
DEBUG:root:Length of F = 180
DEBUG:root:Length of F = 176
DEBUG:root:Length of F = 156
DEBUG:root:Length of F = 148
DEBUG:root:Length of F = 132
DEBUG:root:Length of F = 124
DEBUG:root:Length of F = 116
DEBUG:root:Length of F = 108
DEBUG:root:Length of F = 96
DEBUG:root:Length of F = 92
DEBUG:root:Length of F = 88
DEBUG:root:Length

DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root:Epoch = 19
DEBUG:root:Epoch = 20
DEBUG:root:Epoch = 21
DEBUG:root:Epoch = 22
DEBUG:root:Epoch = 23
DEBUG:root:Epoch = 24
DEBUG:root:Epoch = 25
DEBUG:root:Epoch = 26
DEBUG:root:Epoch = 27
DEBUG:root:Epoch = 28
DEBUG:root:Epoch = 29
DEBUG:root:Epoch = 30
DEBUG:root:Epoch = 31
DEBUG:root:Epoch = 32
DEBUG:root:Epoch = 33
DEBUG:root:Epoch = 34
DEBUG:root:Epoch = 35
DEBUG:root:Epoch = 36
DEBUG:root:Epoch = 37
DEBUG:root:Epoch = 38
DEBUG:root:Epoch = 39
DEBUG:root:Epoch = 40
DEBUG:root:Epoch = 41
DEBUG:root:Epoch = 42
DEBUG:root:Epoch = 43
DEBUG:root:Epoch = 44
DEBUG:root:Epoch = 45
DEBUG:root:Epoch = 46
DEBUG:root:Epoch = 47
DEBUG:root:Epoch = 48
DEBUG:root:Epoch = 49
DEBUG:root:Total epochs = 50
INFO:root:Community 7 proces

...

DEBUG:root:Epoch = 21
DEBUG:root:Epoch = 22
DEBUG:root:Epoch = 23
DEBUG:root:Epoch = 24
DEBUG:root:Epoch = 25
DEBUG:root:Epoch = 26
DEBUG:root:Epoch = 27
DEBUG:root:Epoch = 28
DEBUG:root:Epoch = 29
DEBUG:root:Epoch = 30
DEBUG:root:Epoch = 31
DEBUG:root:Epoch = 32
DEBUG:root:Epoch = 33
DEBUG:root:Epoch = 34
DEBUG:root:Epoch = 35
DEBUG:root:Epoch = 36
DEBUG:root:Epoch = 37
DEBUG:root:Epoch = 38
DEBUG:root:Epoch = 39
DEBUG:root:Epoch = 40
DEBUG:root:Epoch = 41
DEBUG:root:Epoch = 42
DEBUG:root:Epoch = 43
DEBUG:root:Epoch = 44
DEBUG:root:Epoch = 45
DEBUG:root:Epoch = 46
DEBUG:root:Epoch = 47
DEBUG:root:Epoch = 48
DEBUG:root:Epoch = 49
DEBUG:root:Total epochs = 50
INFO:root:Total communities 15
INFO:root:Community 1 processing...
INFO:root:Build network on graph G:Graph with 15 nodes and 35 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:ro

DEBUG:root:Epoch = 36
DEBUG:root:Epoch = 37
DEBUG:root:Epoch = 38
DEBUG:root:Epoch = 39
DEBUG:root:Epoch = 40
DEBUG:root:Epoch = 41
DEBUG:root:Epoch = 42
DEBUG:root:Epoch = 43
DEBUG:root:Epoch = 44
DEBUG:root:Epoch = 45
DEBUG:root:Epoch = 46
DEBUG:root:Epoch = 47
DEBUG:root:Epoch = 48
DEBUG:root:Epoch = 49
DEBUG:root:Total epochs = 50
INFO:root:Community 10 processing...
INFO:root:Build network on graph G:Graph with 16 nodes and 46 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Optimal solution found. Cardinality = 5
DEBUG:root:Total epochs = 2
INFO:root:Community 11 processing...
INFO:root:Build network on graph G:Graph with 17 nodes and 38 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root

DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root:Epoch = 19
DEBUG:root:Epoch = 20
DEBUG:root:Epoch = 21
DEBUG:root:Epoch = 22
DEBUG:root:Epoch = 23
DEBUG:root:Epoch = 24
DEBUG:root:Epoch = 25
DEBUG:root:Epoch = 26
DEBUG:root:Epoch = 27
DEBUG:root:Epoch = 28
DEBUG:root:Epoch = 29
DEBUG:root:Epoch = 30
DEBUG:root:Epoch = 31
DEBUG:root:Epoch = 32
DEBUG:root:Epoch = 33
DEBUG:root:Epoch = 34
DEBUG:root:Optimal solution found. Cardinality = 8
DEBUG:root:Total epochs = 35
INFO:root:Community 4 processing...
INFO:root:Build network on graph G:Graph with 19 nodes and 54 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Ep

DEBUG:root:Epoch = 29
DEBUG:root:Epoch = 30
DEBUG:root:Epoch = 31
DEBUG:root:Epoch = 32
DEBUG:root:Epoch = 33
DEBUG:root:Epoch = 34
DEBUG:root:Epoch = 35
DEBUG:root:Epoch = 36
DEBUG:root:Epoch = 37
DEBUG:root:Epoch = 38
DEBUG:root:Epoch = 39
DEBUG:root:Epoch = 40
DEBUG:root:Epoch = 41
DEBUG:root:Epoch = 42
DEBUG:root:Epoch = 43
DEBUG:root:Epoch = 44
DEBUG:root:Epoch = 45
DEBUG:root:Epoch = 46
DEBUG:root:Epoch = 47
DEBUG:root:Epoch = 48
DEBUG:root:Epoch = 49
DEBUG:root:Total epochs = 50
INFO:root:Replacing forbiden nodes
DEBUG:root:Length of F = 684
DEBUG:root:Length of F = 672
DEBUG:root:Length of F = 636
DEBUG:root:Length of F = 608
DEBUG:root:Length of F = 584
DEBUG:root:Length of F = 560
DEBUG:root:Length of F = 536
DEBUG:root:Length of F = 520
DEBUG:root:Length of F = 492
DEBUG:root:Length of F = 488
DEBUG:root:Length of F = 460
DEBUG:root:Length of F = 448
DEBUG:root:Length of F = 424
DEBUG:root:Length of F = 404
DEBUG:root:Length of F = 376
DEBUG:root:Length of F = 364
DEBUG:root

dNN result on SBM 250, 0.1 = 29.2


In [43]:
test_average_result_on_graph(nx.stochastic_block_model([250],[[0.2]]),"SBM 250, 0.2")

INFO:root:Total communities 14
INFO:root:Community 1 processing...
INFO:root:Build network on graph G:Graph with 20 nodes and 91 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 2 processing...
INFO:root:Build network on graph G:Graph with 26 nodes and 131 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 3 processing...
INFO:root:Build network on graph G:Graph with 16 nodes and 64 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root:Epoch = 19
DEBUG:root:Epoch = 20
DEBUG:root:Op

DEBUG:root:Length of F = 216
DEBUG:root:Length of F = 200
DEBUG:root:Length of F = 172
DEBUG:root:Length of F = 160
DEBUG:root:Length of F = 144
DEBUG:root:Length of F = 128
DEBUG:root:Length of F = 108
DEBUG:root:Length of F = 96
DEBUG:root:Length of F = 88
DEBUG:root:Length of F = 80
DEBUG:root:Length of F = 64
DEBUG:root:Length of F = 48
DEBUG:root:Length of F = 36
DEBUG:root:Length of F = 32
DEBUG:root:Length of F = 28
DEBUG:root:Length of F = 24
DEBUG:root:Length of F = 20
DEBUG:root:Length of F = 16
DEBUG:root:Length of F = 4
INFO:root:Left nodes size = 67
INFO:root:Build network on graph G:Graph with 67 nodes and 437 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 

INFO:root:Community 12 processing...
INFO:root:Build network on graph G:Graph with 8 nodes and 22 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 13 processing...
INFO:root:Build network on graph G:Graph with 23 nodes and 112 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 14 processing...
INFO:root:Build network on graph G:Graph with 13 nodes and 44 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Replacing forbiden nodes
DEBUG:root:Length of F = 84
DEBUG:root:Length of F = 80
DEBUG:root:Length of F = 56
DEBUG:root:Length of F = 48
DEBUG:root:Length of F = 40
DEBUG:root:Length of F = 28
DEBUG:root:Length of F = 24
DEBUG:root:Length of F = 16
DEBUG:root:Length of F = 8
DEBUG:root:Length of F = 4
INFO:root:Left nodes size = 54
INFO:root:Build network on graph G:Graph with 54 nodes a

DEBUG:root:Epoch = 9
DEBUG:root:Optimal solution found. Cardinality = 4
DEBUG:root:Total epochs = 10
INFO:root:Community 13 processing...
INFO:root:Build network on graph G:Graph with 13 nodes and 46 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 14 processing...
INFO:root:Build network on graph G:Graph with 14 nodes and 51 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 15 processing...
INFO:root:Build network on graph G:Graph with 16 nodes and 66 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Optimal solution found. Cardinality = 5
DEBUG:root:Total epochs = 9
INFO:root:Community 16 processing...
INFO:root:Build network on graph G:Graph with 13 nodes and 41 edges.
DEBUG:root:Epoch = 

DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 9 processing...
INFO:root:Build network on graph G:Graph with 24 nodes and 119 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 18
INFO:root:Community 10 processing...
INFO:root:Build network on graph G:Graph with 12 nodes and 40 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13

...

DEBUG:root:Length of F = 148
DEBUG:root:Length of F = 132
DEBUG:root:Length of F = 116
DEBUG:root:Length of F = 104
DEBUG:root:Length of F = 96
DEBUG:root:Length of F = 92
DEBUG:root:Length of F = 76
DEBUG:root:Length of F = 64
DEBUG:root:Length of F = 56
DEBUG:root:Length of F = 48
DEBUG:root:Length of F = 32
DEBUG:root:Length of F = 28
DEBUG:root:Length of F = 24
DEBUG:root:Length of F = 16
DEBUG:root:Length of F = 12
DEBUG:root:Length of F = 4
INFO:root:Left nodes size = 29
INFO:root:Build network on graph G:Graph with 29 nodes and 90 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Optimal solution found. Cardinality = 11
DEBUG:root:

DEBUG:root:Epoch = 28
DEBUG:root:Epoch = 29
DEBUG:root:Epoch = 30
DEBUG:root:Epoch = 31
DEBUG:root:Epoch = 32
DEBUG:root:Epoch = 33
DEBUG:root:Epoch = 34
DEBUG:root:Epoch = 35
DEBUG:root:Epoch = 36
DEBUG:root:Epoch = 37
DEBUG:root:Epoch = 38
DEBUG:root:Epoch = 39
DEBUG:root:Optimal solution found. Cardinality = 4
DEBUG:root:Total epochs = 40
INFO:root:Community 14 processing...
INFO:root:Build network on graph G:Graph with 19 nodes and 80 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Optimal solution found. Cardinality = 5
DEBUG:root:Total epochs = 17
INFO:root:Community 15 processing...
INFO:root:Build network on graph G:Graph with 11 nodes and 33 edges.


DEBUG:root:Epoch = 39
DEBUG:root:Epoch = 40
DEBUG:root:Epoch = 41
DEBUG:root:Epoch = 42
DEBUG:root:Epoch = 43
DEBUG:root:Epoch = 44
DEBUG:root:Epoch = 45
DEBUG:root:Epoch = 46
DEBUG:root:Epoch = 47
DEBUG:root:Epoch = 48
DEBUG:root:Epoch = 49
DEBUG:root:Total epochs = 50
INFO:root:Community 9 processing...
INFO:root:Build network on graph G:Graph with 12 nodes and 38 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Optimal solution found. Cardinality = 4
DEBUG:root:Total epochs = 1
INFO:root:Community 10 processing...
INFO:root:Build network on graph G:Graph with 19 nodes and 84 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root

DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root:Epoch = 19
DEBUG:root:Epoch = 20
DEBUG:root:Epoch = 21
DEBUG:root:Epoch = 22
DEBUG:root:Epoch = 23
DEBUG:root:Epoch = 24
DEBUG:root:Epoch = 25
DEBUG:root:Epoch = 26
DEBUG:root:Epoch = 27
DEBUG:root:Epoch = 28
DEBUG:root:Epoch = 29
DEBUG:root:Epoch = 30
DEBUG:root:Epoch = 31
DEBUG:root:Epoch = 32
DEBUG:root:Epoch = 33
DEBUG:root:Epoch = 34
DEBUG:root:Epoch = 35
DEBUG:root:Epoch = 36
DEBUG:root:Epoch = 37
DEBUG:root:Epoch = 38
DEBUG:root:Epoch = 39
DEBUG:root:Optimal solution found. Cardinality = 6
DEBUG:root:Total epochs = 40
INFO:root:Community 5 processing...
INFO:root:Build network on graph G:Graph with 18 nodes and 74 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 6 processing...
INFO:root:Build network on graph G:Graph with 17 nodes and 68 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epoch

dNN result on SBM 250, 0.2 = 8.8


In [44]:
test_average_result_on_graph(nx.stochastic_block_model([350],[[0.1]]),"SBM 350, 0.1")

INFO:root:Total communities 17
INFO:root:Community 1 processing...
INFO:root:Build network on graph G:Graph with 28 nodes and 102 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root:Epoch = 19
DEBUG:root:Epoch = 20
DEBUG:root:Epoch = 21
DEBUG:root:Epoch = 22
DEBUG:root:Epoch = 23
DEBUG:root:Epoch = 24
DEBUG:root:Epoch = 25
DEBUG:root:Epoch = 26
DEBUG:root:Epoch = 27
DEBUG:root:Epoch = 28
DEBUG:root:Epoch = 29
DEBUG:root:Epoch = 30
DEBUG:root:Epoch = 31
DEBUG:root:Epoch = 32
DEBUG:root:Epoch = 33
DEBUG:root:Epoch = 34
DEBUG:root:Epoch = 35
DEBUG:root:Epoch = 36
DEBUG:root:Epoch = 37
DEBUG:root:Epoch = 38
DEBUG:root:Epoc

DEBUG:root:Epoch = 44
DEBUG:root:Epoch = 45
DEBUG:root:Epoch = 46
DEBUG:root:Epoch = 47
DEBUG:root:Epoch = 48
DEBUG:root:Epoch = 49
DEBUG:root:Total epochs = 50
INFO:root:Community 9 processing...
INFO:root:Build network on graph G:Graph with 29 nodes and 114 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root:Epoch = 19
DEBUG:root:Epoch = 20
DEBUG:root:Epoch = 21
DEBUG:root:Epoch = 22
DEBUG:root:Epoch = 23
DEBUG:root:Epoch = 24
DEBUG:root:Epoch = 25
DEBUG:root:Epoch = 26
DEBUG:root:Epoch = 27
DEBUG:root:Epoch = 28
DEBUG:root:Epoch = 29
DEBUG:root:Epoch = 30
DEBUG:root:Epoch = 31
DEBUG:root:Epoch = 32
DEBUG:root:Epoch 

DEBUG:root:Length of F = 684
DEBUG:root:Length of F = 676
DEBUG:root:Length of F = 668
DEBUG:root:Length of F = 644
DEBUG:root:Length of F = 636
DEBUG:root:Length of F = 632
DEBUG:root:Length of F = 620
DEBUG:root:Length of F = 576
DEBUG:root:Length of F = 572
DEBUG:root:Length of F = 548
DEBUG:root:Length of F = 544
DEBUG:root:Length of F = 536
DEBUG:root:Length of F = 520
DEBUG:root:Length of F = 500
DEBUG:root:Length of F = 484
DEBUG:root:Length of F = 468
DEBUG:root:Length of F = 444
DEBUG:root:Length of F = 432
DEBUG:root:Length of F = 420
DEBUG:root:Length of F = 396
DEBUG:root:Length of F = 388
DEBUG:root:Length of F = 368
DEBUG:root:Length of F = 352
DEBUG:root:Length of F = 324
DEBUG:root:Length of F = 312
DEBUG:root:Length of F = 300
DEBUG:root:Length of F = 292
DEBUG:root:Length of F = 284
DEBUG:root:Length of F = 272
DEBUG:root:Length of F = 260
DEBUG:root:Length of F = 244
DEBUG:root:Length of F = 240
DEBUG:root:Length of F = 228
DEBUG:root:Length of F = 220
DEBUG:root:Len

DEBUG:root:Epoch = 29
DEBUG:root:Epoch = 30
DEBUG:root:Epoch = 31
DEBUG:root:Epoch = 32
DEBUG:root:Epoch = 33
DEBUG:root:Epoch = 34
DEBUG:root:Epoch = 35
DEBUG:root:Epoch = 36
DEBUG:root:Epoch = 37
DEBUG:root:Epoch = 38
DEBUG:root:Epoch = 39
DEBUG:root:Epoch = 40
DEBUG:root:Epoch = 41
DEBUG:root:Epoch = 42
DEBUG:root:Epoch = 43
DEBUG:root:Optimal solution found. Cardinality = 7
DEBUG:root:Total epochs = 44
INFO:root:Community 7 processing...
INFO:root:Build network on graph G:Graph with 20 nodes and 73 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Optimal solution found. Cardinality = 6
DEBUG:root:Total epochs = 11
INFO:root:Community 8 processing...
INFO:root:Build network on graph G:Graph with 15 nodes and 40 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG

DEBUG:root:Epoch = 32
DEBUG:root:Epoch = 33
DEBUG:root:Epoch = 34
DEBUG:root:Epoch = 35
DEBUG:root:Epoch = 36
DEBUG:root:Epoch = 37
DEBUG:root:Epoch = 38
DEBUG:root:Epoch = 39
DEBUG:root:Epoch = 40
DEBUG:root:Epoch = 41
DEBUG:root:Epoch = 42
DEBUG:root:Epoch = 43
DEBUG:root:Epoch = 44
DEBUG:root:Epoch = 45
DEBUG:root:Epoch = 46
DEBUG:root:Epoch = 47
DEBUG:root:Optimal solution found. Cardinality = 8
DEBUG:root:Total epochs = 48
INFO:root:Community 15 processing...
INFO:root:Build network on graph G:Graph with 24 nodes and 81 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root:Epoch = 19
DEBUG:root:Epoch = 20
DEBUG:root

...

DEBUG:root:Epoch = 19
DEBUG:root:Epoch = 20
DEBUG:root:Epoch = 21
DEBUG:root:Epoch = 22
DEBUG:root:Epoch = 23
DEBUG:root:Epoch = 24
DEBUG:root:Epoch = 25
DEBUG:root:Epoch = 26
DEBUG:root:Epoch = 27
DEBUG:root:Epoch = 28
DEBUG:root:Epoch = 29
DEBUG:root:Epoch = 30
DEBUG:root:Epoch = 31
DEBUG:root:Epoch = 32
DEBUG:root:Epoch = 33
DEBUG:root:Epoch = 34
DEBUG:root:Epoch = 35
DEBUG:root:Epoch = 36
DEBUG:root:Epoch = 37
DEBUG:root:Epoch = 38
DEBUG:root:Epoch = 39
DEBUG:root:Epoch = 40
DEBUG:root:Epoch = 41
DEBUG:root:Epoch = 42
DEBUG:root:Epoch = 43
DEBUG:root:Epoch = 44
DEBUG:root:Epoch = 45
DEBUG:root:Epoch = 46
DEBUG:root:Epoch = 47
DEBUG:root:Epoch = 48
DEBUG:root:Epoch = 49
DEBUG:root:Total epochs = 50
INFO:root:Community 8 processing...
INFO:root:Build network on graph G:Graph with 23 nodes and 79 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch

INFO:root:Community 17 processing...
INFO:root:Build network on graph G:Graph with 25 nodes and 80 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root:Epoch = 19
DEBUG:root:Epoch = 20
DEBUG:root:Epoch = 21
DEBUG:root:Epoch = 22
DEBUG:root:Epoch = 23
DEBUG:root:Epoch = 24
DEBUG:root:Epoch = 25
DEBUG:root:Epoch = 26
DEBUG:root:Epoch = 27
DEBUG:root:Epoch = 28
DEBUG:root:Epoch = 29
DEBUG:root:Epoch = 30
DEBUG:root:Epoch = 31
DEBUG:root:Epoch = 32
DEBUG:root:Epoch = 33
DEBUG:root:Epoch = 34
DEBUG:root:Epoch = 35
DEBUG:root:Epoch = 36
DEBUG:root:Epoch = 37
DEBUG:root:Epoch = 38
DEBUG:root:Epoch = 39
DEBUG:root:Epoch = 40
DE

DEBUG:root:Epoch = 40
DEBUG:root:Epoch = 41
DEBUG:root:Epoch = 42
DEBUG:root:Epoch = 43
DEBUG:root:Epoch = 44
DEBUG:root:Epoch = 45
DEBUG:root:Epoch = 46
DEBUG:root:Epoch = 47
DEBUG:root:Epoch = 48
DEBUG:root:Epoch = 49
DEBUG:root:Total epochs = 50
INFO:root:Community 5 processing...
INFO:root:Build network on graph G:Graph with 10 nodes and 24 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root:Epoch = 19
DEBUG:root:Epoch = 20
DEBUG:root:Epoch = 21
DEBUG:root:Epoch = 22
DEBUG:root:Optimal solution found. Cardinality = 4
DEBUG:root:Total epochs = 23
INFO:root:Community 6 processing...
INFO:root:Build network on graph G

INFO:root:Community 14 processing...
INFO:root:Build network on graph G:Graph with 14 nodes and 42 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Optimal solution found. Cardinality = 4
DEBUG:root:Total epochs = 7
INFO:root:Community 15 processing...
INFO:root:Build network on graph G:Graph with 18 nodes and 51 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root:Epoch = 19
DEBUG:root:Epoch = 20
DEBUG:root:Epoch = 21
DEBUG:root:Epoch = 22
DEBUG:root:Epoch = 23
DEBUG:root:Epoch = 24
DEBUG:root:Epoch = 25


dNN result on SBM 350, 0.1 = 20.5


In [45]:
test_average_result_on_graph(nx.stochastic_block_model([350],[[0.2]]),"SBM 350, 0.2")

INFO:root:Total communities 16
INFO:root:Community 1 processing...
INFO:root:Build network on graph G:Graph with 19 nodes and 83 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 2 processing...
INFO:root:Build network on graph G:Graph with 18 nodes and 84 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 3 processing...
INFO:root:Build network on graph G:Graph with 30 nodes and 184 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 4 processing...
INFO:root:Build network on graph G:Graph with 22 nodes and 108 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 5 processing...
INFO:root:Build network on graph G:Graph with 25 nodes and 131 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck

DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Optimal solution found. Cardinality = 5
DEBUG:root:Total epochs = 14
INFO:root:Community 3 processing...
INFO:root:Build network on graph G:Graph with 28 nodes and 163 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root:Epoch = 19
DEBUG:root:Epoch = 20
DEBUG:root:Epoch = 21
DEBUG:root:Epoch = 22
DEBUG:root:Epoch = 23
DEBUG:root:Epoch = 24
DEBUG:root:Epoch = 25
DEBUG:root:Epoch = 26
DEBUG:root:Epoch = 27
DEBUG:root:Epoch = 28
DEBUG:root:Epo

DEBUG:root:Epoch = 43
DEBUG:root:Epoch = 44
DEBUG:root:Epoch = 45
DEBUG:root:Epoch = 46
DEBUG:root:Epoch = 47
DEBUG:root:Epoch = 48
DEBUG:root:Epoch = 49
DEBUG:root:Total epochs = 50
INFO:root:Total communities 18
INFO:root:Community 1 processing...
INFO:root:Build network on graph G:Graph with 20 nodes and 91 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 2 processing...
INFO:root:Build network on graph G:Graph with 29 nodes and 168 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root:Epoch = 19
DEBUG:root:Epoch = 20
DEBUG:root:Epoch = 21
DEBUG:r

DEBUG:root:Epoch = 35
DEBUG:root:Epoch = 36
DEBUG:root:Epoch = 37
DEBUG:root:Epoch = 38
DEBUG:root:Epoch = 39
DEBUG:root:Epoch = 40
DEBUG:root:Epoch = 41
DEBUG:root:Epoch = 42
DEBUG:root:Epoch = 43
DEBUG:root:Epoch = 44
DEBUG:root:Epoch = 45
DEBUG:root:Epoch = 46
DEBUG:root:Epoch = 47
DEBUG:root:Epoch = 48
DEBUG:root:Epoch = 49
DEBUG:root:Total epochs = 50
INFO:root:Community 15 processing...
INFO:root:Build network on graph G:Graph with 29 nodes and 174 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root:Epoch = 19
DEBUG:root:Epoch = 20
DEBUG:root:Epoch = 21
DEBUG:root:Epoch = 22
DEBUG:root:Epoch = 23
DEBUG:root:Epoch

DEBUG:root:Epoch = 20
DEBUG:root:Epoch = 21
DEBUG:root:Epoch = 22
DEBUG:root:Epoch = 23
DEBUG:root:Epoch = 24
DEBUG:root:Epoch = 25
DEBUG:root:Epoch = 26
DEBUG:root:Epoch = 27
DEBUG:root:Epoch = 28
DEBUG:root:Epoch = 29
DEBUG:root:Epoch = 30
DEBUG:root:Epoch = 31
DEBUG:root:Epoch = 32
DEBUG:root:Epoch = 33
DEBUG:root:Epoch = 34
DEBUG:root:Epoch = 35
DEBUG:root:Epoch = 36
DEBUG:root:Epoch = 37
DEBUG:root:Epoch = 38
DEBUG:root:Epoch = 39
DEBUG:root:Epoch = 40
DEBUG:root:Epoch = 41
DEBUG:root:Epoch = 42
DEBUG:root:Epoch = 43
DEBUG:root:Epoch = 44
DEBUG:root:Epoch = 45
DEBUG:root:Epoch = 46
DEBUG:root:Epoch = 47
DEBUG:root:Epoch = 48
DEBUG:root:Epoch = 49
DEBUG:root:Total epochs = 50
INFO:root:Community 9 processing...
INFO:root:Build network on graph G:Graph with 20 nodes and 94 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 10 processing...
INFO:root:Build network on graph G:Graph with 23 nodes and 113 edges.
DEBUG

...

DEBUG:root:Optimal solution found. Cardinality = 6
DEBUG:root:Total epochs = 4
INFO:root:Community 7 processing...
INFO:root:Build network on graph G:Graph with 21 nodes and 100 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root:Epoch = 19
DEBUG:root:Epoch = 20
DEBUG:root:Epoch = 21
DEBUG:root:Epoch = 22
DEBUG:root:Epoch = 23
DEBUG:root:Epoch = 24
DEBUG:root:Epoch = 25
DEBUG:root:Epoch = 26
DEBUG:root:Epoch = 27
DEBUG:root:Epoch = 28
DEBUG:root:Epoch = 29
DEBUG:root:Epoch = 30
DEBUG:root:Epoch = 31
DEBUG:root:Epoch = 32
DEBUG:root:Epoch = 33
DEBUG:root:Epoch = 34
DEBUG:root:Epoch = 35
DEBUG:root:Epoch = 36
DEBUG:root:

DEBUG:root:Epoch = 27
DEBUG:root:Optimal solution found. Cardinality = 3
DEBUG:root:Total epochs = 28
INFO:root:Community 18 processing...
INFO:root:Build network on graph G:Graph with 21 nodes and 101 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root:Epoch = 19
DEBUG:root:Epoch = 20
DEBUG:root:Epoch = 21
DEBUG:root:Epoch = 22
DEBUG:root:Epoch = 23
DEBUG:root:Epoch = 24
DEBUG:root:Epoch = 25
DEBUG:root:Epoch = 26
DEBUG:root:Epoch = 27
DEBUG:root:Epoch = 28
DEBUG:root:Epoch = 29
DEBUG:root:Epoch = 30
DEBUG:root:Epoch = 31
DEBUG:root:Epoch = 32
DEBUG:root:Epoch = 33
DEBUG:root:Epoch = 34
DEBUG:root:Epoch = 35
DEBUG:roo

DEBUG:root:Total epochs = 12
INFO:root:Community 8 processing...
INFO:root:Build network on graph G:Graph with 29 nodes and 171 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 9 processing...
INFO:root:Build network on graph G:Graph with 21 nodes and 95 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root:Epoch = 19
DEBUG:root:Epoch = 20
DEBUG:root:Epoch = 21
DEBUG:root:Epoch = 22
DEBUG:root:Epoch = 23
DEBUG:root:Epoch = 24
DEBUG:root:Epoch = 25
DEBUG:root:Epoch = 26
DEBUG:root:Epoch = 27
DEBUG:root:Epoch = 28
DEBUG:root:Epoch = 29
DEBUG:root:Epoch

DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root:Epoch = 19
DEBUG:root:Epoch = 20
DEBUG:root:Epoch = 21
DEBUG:root:Epoch = 22
DEBUG:root:Epoch = 23
DEBUG:root:Epoch = 24
DEBUG:root:Epoch = 25
DEBUG:root:Epoch = 26
DEBUG:root:Epoch = 27
DEBUG:root:Epoch = 28
DEBUG:root:Epoch = 29
DEBUG:root:Epoch = 30
DEBUG:root:Epoch = 31
DEBUG:root:Epoch = 32
DEBUG:root:Epoch = 33
DEBUG:root:Epoch = 34
DEBUG:root:Epoch = 35
DEBUG:root:Epoch = 36
DEBUG:root:Epoch = 37
DEBUG:root:Epoch = 38
DEBUG:root:Epoch = 39
DEBUG:root:Epoch = 40
DEBUG:root:Epoch = 41
DEBUG:root:Epoch = 42
DEBUG:root:Epoch = 43
DEBUG:root:Epoch = 44
DEBUG:root:Epoch = 4

dNN result on SBM 350, 0.2 = 6.6


In [55]:
test_average_result_on_graph(nx.barabasi_albert_graph(100,50),"BA 100")

INFO:root:Total communities 15
INFO:root:Community 1 processing...
INFO:root:Build network on graph G:Graph with 1 nodes and 0 edges.
DEBUG:root:Optimal solution found. Cardinality = 1
DEBUG:root:Total epochs = 0
INFO:root:Community 2 processing...
INFO:root:Build network on graph G:Graph with 8 nodes and 22 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 3 processing...
INFO:root:Build network on graph G:Graph with 8 nodes and 22 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 4 processing...
INFO:root:Build network on graph G:Graph with 9 nodes and 26 edges.
DEBUG:root:Optimal solution found. Cardinality = 5
DEBUG:root:Total epochs = 0
INFO:root:Community 5 processing...
INFO:root:Build network on graph G:Graph with 9 nodes and 26 edges.
DEBUG:root:Optimal solution found. Cardinality = 5
DEBUG:root:Total epochs = 0
INFO:root:Community 6 pro

DEBUG:root:Length of F = 100
DEBUG:root:Length of F = 88
DEBUG:root:Length of F = 76
DEBUG:root:Length of F = 64
DEBUG:root:Length of F = 52
DEBUG:root:Length of F = 44
DEBUG:root:Length of F = 36
DEBUG:root:Length of F = 24
DEBUG:root:Length of F = 12
DEBUG:root:Length of F = 4
INFO:root:Left nodes size = 10
INFO:root:Build network on graph G:Graph with 10 nodes and 18 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root:Epoch = 19
DEBUG:root:Epoch = 20
DEBUG:root:Epoch = 21
DEBUG:root:Epoch = 22
DEBUG:root:Epoch = 23
DEBUG:root:Epoch = 24
DEBUG:root:Epoch = 25
DEBUG:root:Epoch = 26
DEBUG:root:Epoch = 27
DEBUG:root:Epo

INFO:root:Build network on graph G:Graph with 9 nodes and 24 edges.
DEBUG:root:Optimal solution found. Cardinality = 5
DEBUG:root:Total epochs = 0
INFO:root:Community 13 processing...
INFO:root:Build network on graph G:Graph with 10 nodes and 33 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Optimal solution found. Cardinality = 5
DEBUG:root:Total epochs = 1
INFO:root:Community 14 processing...
INFO:root:Build network on graph G:Graph with 5 nodes and 9 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 15 processing...
INFO:root:Build network on graph G:Graph with 4 nodes and 5 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 16 processing...
INFO:root:Build network on graph G:Graph with 1 nodes and 0 edges.
DEBUG:root:Optimal solution found. Cardinality = 1
DEBUG:root:Total epochs = 0
INFO:root:Community 17 processing...
INFO:root:Build network on grap

DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root:Epoch = 19
DEBUG:root:Epoch = 20
DEBUG:root:Epoch = 21
DEBUG:root:Epoch = 22
DEBUG:root:Epoch = 23
DEBUG:root:Epoch = 24
DEBUG:root:Epoch = 25
DEBUG:root:Epoch = 26
DEBUG:root:Epoch = 27
DEBUG:root:Epoch = 28
DEBUG:root:Epoch = 29
DEBUG:root:Epoch = 30
DEBUG:root:Epoch = 31
DEBUG:root:Epoch = 32
DEBUG:root:Epoch = 33
DEBUG:root:Epoch = 34
DEBUG:root:Epoch = 35
DEBUG:root:Epoch = 36
DEBUG:root:Epoch = 37
DEBUG:root:Optimal solution found. Cardinality = 4
DEBUG:root:Total epochs = 38
INFO:root:Total communities 15
INFO:root:Community 1 processing...
INFO:root:Build network on graph G:Graph with 1 nodes and 0 edges.
DEBUG:root:Optimal solution found. Cardinality = 1
DEBUG:root:Total epochs = 0
INFO:root:Community 2 processing...
INFO:root:Build network on graph G:Graph with 7 nodes a

INFO:root:Community 13 processing...
INFO:root:Build network on graph G:Graph with 10 nodes and 30 edges.
DEBUG:root:Optimal solution found. Cardinality = 6
DEBUG:root:Total epochs = 0
INFO:root:Community 14 processing...
INFO:root:Build network on graph G:Graph with 1 nodes and 0 edges.
DEBUG:root:Optimal solution found. Cardinality = 1
DEBUG:root:Total epochs = 0
INFO:root:Community 15 processing...
INFO:root:Build network on graph G:Graph with 1 nodes and 0 edges.
DEBUG:root:Optimal solution found. Cardinality = 1
DEBUG:root:Total epochs = 0
INFO:root:Replacing forbiden nodes
DEBUG:root:Length of F = 468
DEBUG:root:Length of F = 304
DEBUG:root:Length of F = 296
DEBUG:root:Length of F = 288
DEBUG:root:Length of F = 280
DEBUG:root:Length of F = 272
DEBUG:root:Length of F = 264
DEBUG:root:Length of F = 256
DEBUG:root:Length of F = 248
DEBUG:root:Length of F = 244
DEBUG:root:Length of F = 236
DEBUG:root:Length of F = 232
DEBUG:root:Length of F = 224
DEBUG:root:Length of F = 216
DEBUG:ro

DEBUG:root:Epoch = 45
DEBUG:root:Epoch = 46
DEBUG:root:Epoch = 47
DEBUG:root:Epoch = 48
DEBUG:root:Epoch = 49
DEBUG:root:Total epochs = 50
INFO:root:Total communities 15
INFO:root:Community 1 processing...
INFO:root:Build network on graph G:Graph with 6 nodes and 12 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 2 processing...
INFO:root:Build network on graph G:Graph with 12 nodes and 46 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Optimal solution found. Cardinality = 6
DEBUG:root:Total epochs = 1
INFO:root:Community 3 processing...
INFO:root:Build network on graph G:Graph with 11 nodes and 39 edges.
DEBUG:root:Optimal solution found. Cardinality = 6
DEBUG:root:Total epochs = 0
INFO:root:Community 4 processing...
INFO:root:Build network on graph G:Graph with 10 nodes and 35 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 5 processing...
INFO:roo

DEBUG:root:Length of F = 32
DEBUG:root:Length of F = 24
DEBUG:root:Length of F = 20
DEBUG:root:Length of F = 12
DEBUG:root:Length of F = 4
INFO:root:Left nodes size = 5
INFO:root:Build network on graph G:Graph with 5 nodes and 0 edges.
DEBUG:root:Optimal solution found. Cardinality = 5
DEBUG:root:Total epochs = 0


dNN result on BA 100 = 6.7


In [56]:
test_average_result_on_graph(nx.barabasi_albert_graph(200,90),"BA 200")

INFO:root:Total communities 27
INFO:root:Community 1 processing...
INFO:root:Build network on graph G:Graph with 1 nodes and 0 edges.
DEBUG:root:Optimal solution found. Cardinality = 1
DEBUG:root:Total epochs = 0
INFO:root:Community 2 processing...
INFO:root:Build network on graph G:Graph with 11 nodes and 42 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Optimal solution found. Cardinality = 5
DEBUG:root:Total epochs = 1
INFO:root:Community 3 processing...
INFO:root:Build network on graph G:Graph with 8 nodes and 22 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 4 processing...
INFO:root:Build network on graph G:Graph with 10 nodes and 34 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 5 processing...
INFO:root:Build network on graph G:Graph with 8 nodes and 22 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Tota

DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 7 processing...
INFO:root:Build network on graph G:Graph with 15 nodes and 74 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Optimal solution found. Cardinality = 7
DEBUG:root:Total epochs = 1
INFO:root:Community 8 processing...
INFO:root:Build network on graph G:Graph with 13 nodes and 62 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Optimal solution found. Cardinality = 5
DEBUG:root:Total epochs = 5
INFO:root:Community 9 processing...
INFO:root:Build network on graph G:Graph with 4 nodes and 5 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 10 processing...
INFO:root:Build network on graph G:Graph with 11 nodes and 44 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 11 processing...
INFO:root:Build netw

DEBUG:root:Optimal solution found. Cardinality = 6
DEBUG:root:Total epochs = 3
INFO:root:Community 10 processing...
INFO:root:Build network on graph G:Graph with 14 nodes and 65 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Optimal solution found. Cardinality = 7
DEBUG:root:Total epochs = 1
INFO:root:Community 11 processing...
INFO:root:Build network on graph G:Graph with 6 nodes and 12 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 12 processing...
INFO:root:Build network on graph G:Graph with 11 nodes and 43 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Optimal solution found. Cardinality = 5
DEBUG:root:Total epochs = 3
INFO:root:Community 13 processing...
INFO:root:Build network on graph G:Graph with 13 nodes and 56 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Optimal solution found. Cardinality = 6
DEBUG:root:Total epochs = 1
INFO:root:Community 14 processing...
INFO:root:Build network o

DEBUG:root:Optimal solution found. Cardinality = 6
DEBUG:root:Total epochs = 0
INFO:root:Community 12 processing...
INFO:root:Build network on graph G:Graph with 6 nodes and 12 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 13 processing...
INFO:root:Build network on graph G:Graph with 16 nodes and 92 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 14 processing...
INFO:root:Build network on graph G:Graph with 10 nodes and 34 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Optimal solution found. Cardinality = 5
DEBUG:root:Total epochs = 1
INFO:root:Community 15 processing...
INFO:root:Build network on graph G:Graph with 13 nodes and 60 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Optimal solution found. Cardinality = 6
DEBUG:root:Total epochs = 3
INFO:root:Community 16 processing...
INFO:root:Build network on grap

DEBUG:root:Total epochs = 1
INFO:root:Community 14 processing...
INFO:root:Build network on graph G:Graph with 11 nodes and 41 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Optimal solution found. Cardinality = 5
DEBUG:root:Total epochs = 1
INFO:root:Community 15 processing...
INFO:root:Build network on graph G:Graph with 12 nodes and 48 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Optimal solution found. Cardinality = 6
DEBUG:root:Total epochs = 1
INFO:root:Community 16 processing...
INFO:root:Build network on graph G:Graph with 6 nodes and 12 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 17 processing...
INFO:root:Build network on graph G:Graph with 1 nodes and 0 edges.
DEBUG:root:Optimal solution found. Cardinality = 1
DEBUG:root:Total epochs = 0
INFO:root:Community 18 processing...
INFO:root:Build network on graph G:Graph with 1 nodes and 0 edges.
DEBUG:root:Optimal solution found. Cardinality = 1
DEBUG:root:Total epochs

...

DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 10 processing...
INFO:root:Build network on graph G:Graph with 11 nodes and 44 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Optimal solution found. Cardinality = 5
DEBUG:root:Total epochs = 3
INFO:root:Community 11 processing...
INFO:root:Build network on graph G:Graph with 12 nodes and 48 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Optimal solution found. Cardinality = 6
DEBUG:root:Total epochs = 1
INFO:root:Community 12 processing...
INFO:root:Build network on graph G:Graph with 12 nodes and 50 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Optimal solution found. Cardinality = 6
DEBUG:root:Total epochs = 1
INFO:root:Community 13 processing...
INFO:root:Build network on graph G:Graph with 12 nodes and 53 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Optimal solution found. Cardinality = 5
DEBUG:root:Total epochs = 3
INFO:root:Community

INFO:root:Community 10 processing...
INFO:root:Build network on graph G:Graph with 12 nodes and 53 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 11 processing...
INFO:root:Build network on graph G:Graph with 15 nodes and 77 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Optimal solution found. Cardinality = 7
DEBUG:root:Total epochs = 1
INFO:root:Community 12 processing...
INFO:root:Build network on graph G:Graph with 13 nodes and 54 edges.
DEBUG:root:Optimal solution found. Cardinality = 7
DEBUG:root:Total epochs = 0
INFO:root:Community 13 processing...
INFO:root:Build network on graph G:Graph with 16 nodes and 81 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Optimal solution found. Cardinality = 8
DEBUG:root:Total epochs = 1
INFO:root:Community 14 processing...
INFO:root:Build network on graph G:Graph with 13 nodes and 59 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Optimal solution found.

DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 11 processing...
INFO:root:Build network on graph G:Graph with 16 nodes and 88 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Optimal solution found. Cardinality = 7
DEBUG:root:Total epochs = 3
INFO:root:Community 12 processing...
INFO:root:Build network on graph G:Graph with 14 nodes and 68 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Optimal solution found. Cardinality = 7
DEBUG:root:Total epochs = 1
INFO:root:Community 13 processing...
INFO:root:Build network on graph G:Graph with 16 nodes and 88 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Optimal solution found. Cardinality = 7
DEBUG:root:Total epochs = 3
INFO:root:Community 14 processing...
INFO:root:Build network on graph G:Graph with 12 nodes and 49 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Optimal solution found. Cardinality = 6
DEBUG:root:Total epochs = 1
INFO:root:Community

DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 13 processing...
INFO:root:Build network on graph G:Graph with 13 nodes and 59 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Optimal solution found. Cardinality = 6
DEBUG:root:Total epochs = 3
INFO:root:Community 14 processing...
INFO:root:Build network on graph G:Graph with 9 nodes and 30 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 15 processing...
INFO:root:Build network on graph G:Graph with 11 nodes and 39 edges.
DEBUG:root:Optimal solution found. Cardinality = 6
DEBUG:root:Total epochs = 0
INFO:root:Community 16 processing...
INFO:root:Build network on graph G:Graph with 9 nodes and 30 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 17 processing...
INFO:root:Build network on graph G:Gra

dNN result on BA 200 = 18.7


In [58]:
test_average_result_on_graph(nx.powerlaw_cluster_graph(100,30,0.5),"HK 100")

INFO:root:Total communities 12
INFO:root:Community 1 processing...
INFO:root:Build network on graph G:Graph with 9 nodes and 30 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 2 processing...
INFO:root:Build network on graph G:Graph with 10 nodes and 32 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Optimal solution found. Cardinality = 4
DEBUG:root:Total epochs = 13
INFO:root:Community 3 processing...
INFO:root:Build network on graph G:Graph with 9 nodes and 29 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 4 processing...
INFO:root:Build network on graph G:Graph with 4 nodes and 6 edges.
DEBUG:roo

DEBUG:root:Epoch = 27
DEBUG:root:Epoch = 28
DEBUG:root:Epoch = 29
DEBUG:root:Epoch = 30
DEBUG:root:Epoch = 31
DEBUG:root:Epoch = 32
DEBUG:root:Epoch = 33
DEBUG:root:Epoch = 34
DEBUG:root:Epoch = 35
DEBUG:root:Epoch = 36
DEBUG:root:Epoch = 37
DEBUG:root:Epoch = 38
DEBUG:root:Epoch = 39
DEBUG:root:Epoch = 40
DEBUG:root:Epoch = 41
DEBUG:root:Epoch = 42
DEBUG:root:Epoch = 43
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 44
INFO:root:Community 2 processing...
INFO:root:Build network on graph G:Graph with 8 nodes and 25 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 3 processing...
INFO:root:Build network on graph G:Graph with 12 nodes and 46 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Optimal solution found. Cardinality = 3
DEBUG:root:Total epochs = 3
INFO:root:Community 4 processing...
INFO:root:Build network on graph G:Graph with 16 nodes and 74 edges.
DEBUG:root:Epoch = 0
DE

DEBUG:root:Epoch = 29
DEBUG:root:Epoch = 30
DEBUG:root:Epoch = 31
DEBUG:root:Epoch = 32
DEBUG:root:Epoch = 33
DEBUG:root:Epoch = 34
DEBUG:root:Epoch = 35
DEBUG:root:Epoch = 36
DEBUG:root:Epoch = 37
DEBUG:root:Epoch = 38
DEBUG:root:Epoch = 39
DEBUG:root:Epoch = 40
DEBUG:root:Epoch = 41
DEBUG:root:Epoch = 42
DEBUG:root:Epoch = 43
DEBUG:root:Epoch = 44
DEBUG:root:Epoch = 45
DEBUG:root:Epoch = 46
DEBUG:root:Epoch = 47
DEBUG:root:Epoch = 48
DEBUG:root:Epoch = 49
DEBUG:root:Total epochs = 50
INFO:root:Community 8 processing...
INFO:root:Build network on graph G:Graph with 13 nodes and 49 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Optimal

DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Optimal solution found. Cardinality = 4
DEBUG:root:Total epochs = 8
INFO:root:Community 7 processing...
INFO:root:Build network on graph G:Graph with 11 nodes and 39 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 8 processing...
INFO:root:Build network on graph G:Graph with 7 nodes and 16 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Optimal solution found. Cardinality = 3
DEBUG:root:Total epochs = 16
INFO:root:Community 9 processing...
INFO:root:Bui

DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root:Epoch = 19
DEBUG:root:Epoch = 20
DEBUG:root:Epoch = 21
DEBUG:root:Epoch = 22
DEBUG:root:Epoch = 23
DEBUG:root:Epoch = 24
DEBUG:root:Epoch = 25
DEBUG:root:Epoch = 26
DEBUG:root:Epoch = 27
DEBUG:root:Epoch = 28
DEBUG:root:Epoch = 29
DEBUG:root:Epoch = 30
DEBUG:root:Epoch = 31
DEBUG:root:Epoch = 32
DEBUG:root:Epoch = 33
DEBUG:root:Epoch = 34
DEBUG:root:Epoch = 35
DEBUG:root:Epoch = 36
DEBUG:root:Epoch = 37
DEBUG:root:Epoch = 38
DEBUG:root:Epoch = 39
DEBUG:root:Epoch = 40
DEBUG:root:Epoch = 41
DEBUG:root:Epoch = 42
DEBUG:root:Epoch = 43
DEBUG:root:Epoch = 44
DEBUG:root:Epoch = 4

DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Optimal solution found. Cardinality = 4
DEBUG:root:Total epochs = 3
INFO:root:Community 2 processing...
INFO:root:Build network on graph G:Graph with 7 nodes and 19 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 3 processing...
INFO:root:Build network on graph G:Graph with 10 nodes and 33 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 4 processing...
INFO:root:Build network on graph G:Graph with 12 nodes and 47 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Optimal solution found. Cardinality = 4
DEBUG:root:Total epochs = 3
INFO:root:Community 5 processing...
INFO:root:Build network on graph G:Graph with 6 nodes and 12 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 6 processing...
I

dNN result on HK 100 = 17.4


In [79]:
test_average_result_on_graph(nx.powerlaw_cluster_graph(200,60,0.5),"HK 200", 5)

INFO:root:Total communities 17
INFO:root:Community 1 processing...
INFO:root:Build network on graph G:Graph with 9 nodes and 31 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 2 processing...
INFO:root:Build network on graph G:Graph with 13 nodes and 63 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 3 processing...
INFO:root:Build network on graph G:Graph with 15 nodes and 74 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Optimal solution found. Cardinality = 4
DEBUG:root:Total epochs = 5
INFO:root:Community 4 processing...
INFO:root:Build network on graph G:Graph with 20 nodes and 119 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEBUG:root:Total epochs = 2
INFO:root:Community 5 processing...
INFO:root:Build network on graph G:Graph with 1

DEBUG:root:Epoch = 4
DEBUG:root:Optimal solution found. Cardinality = 5
DEBUG:root:Total epochs = 5
INFO:root:Community 15 processing...
INFO:root:Build network on graph G:Graph with 1 nodes and 0 edges.
DEBUG:root:Optimal solution found. Cardinality = 1
DEBUG:root:Total epochs = 0
INFO:root:Community 16 processing...
INFO:root:Build network on graph G:Graph with 1 nodes and 0 edges.
DEBUG:root:Optimal solution found. Cardinality = 1
DEBUG:root:Total epochs = 0
INFO:root:Replacing forbiden nodes
DEBUG:root:Length of F = 248
DEBUG:root:Length of F = 156
DEBUG:root:Length of F = 76
DEBUG:root:Length of F = 36
INFO:root:Left nodes size = 40
INFO:root:Build network on graph G:Graph with 40 nodes and 0 edges.
DEBUG:root:Optimal solution found. Cardinality = 40
DEBUG:root:Total epochs = 0
INFO:root:Total communities 14
INFO:root:Community 1 processing...
INFO:root:Build network on graph G:Graph with 20 nodes and 126 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Solver stuck
DEB

DEBUG:root:Total epochs = 0
INFO:root:Total communities 16
INFO:root:Community 1 processing...
INFO:root:Build network on graph G:Graph with 25 nodes and 185 edges.
DEBUG:root:Epoch = 0
DEBUG:root:Epoch = 1
DEBUG:root:Epoch = 2
DEBUG:root:Epoch = 3
DEBUG:root:Epoch = 4
DEBUG:root:Epoch = 5
DEBUG:root:Epoch = 6
DEBUG:root:Epoch = 7
DEBUG:root:Epoch = 8
DEBUG:root:Epoch = 9
DEBUG:root:Epoch = 10
DEBUG:root:Epoch = 11
DEBUG:root:Epoch = 12
DEBUG:root:Epoch = 13
DEBUG:root:Epoch = 14
DEBUG:root:Epoch = 15
DEBUG:root:Epoch = 16
DEBUG:root:Epoch = 17
DEBUG:root:Epoch = 18
DEBUG:root:Epoch = 19
DEBUG:root:Epoch = 20
DEBUG:root:Epoch = 21
DEBUG:root:Epoch = 22
DEBUG:root:Epoch = 23
DEBUG:root:Epoch = 24
DEBUG:root:Epoch = 25
DEBUG:root:Epoch = 26
DEBUG:root:Epoch = 27
DEBUG:root:Epoch = 28
DEBUG:root:Epoch = 29
DEBUG:root:Epoch = 30
DEBUG:root:Epoch = 31
DEBUG:root:Epoch = 32
DEBUG:root:Epoch = 33
DEBUG:root:Epoch = 34
DEBUG:root:Epoch = 35
DEBUG:root:Epoch = 36
DEBUG:root:Epoch = 37
DEBUG:roo

dNN result on HK 200 = 47.0
